# 0 IMPORTS

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.core.display import HTML

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## 0.1. FUNCTIONS

In [2]:
def jupyter_settings():
    %matplotlib inline
    
    plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

def describe_num(num_attributes):
    a = num_attributes.describe().T.reset_index()
    
    d4 = pd.DataFrame(num_attributes.apply( lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_attributes.apply( lambda x: x.skew())).T      
    d6 = pd.DataFrame(num_attributes.apply( lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(num_attributes.apply( lambda x: x.unique().shape[0])).T
    
    n = pd.concat([d4, d5, d6, d7], axis=0).T.reset_index()
    n.columns = ['attibutes', 'range', 'skew', 'kurtosis', 'unique']
    b = a.merge(n, right_on='attibutes', left_on='index')
    b.drop('index', axis=1, inplace=True)
    columns = ['attibutes', 'count', 'mean', 'std', '25%', '50%', '75%', 'min', 'max', 'range',
           'skew', 'kurtosis', 'unique']
    return b[columns]

pd.options.display.float_format = '{:.2f}'.format

In [3]:
def analise_brasil(df, variaveis):
    df_base = pd.DataFrame(['AM_LEGAL','1 - Norte','2 - Nordeste','3 - Sudeste','4 - Sul','5 - Centro-Oeste'], columns=['Regiao'])
    for variavel in variaveis:
        a = df[df[variavel]=='Sim'].copy()
        a_ = a[['AM_LEGAL',variavel]].groupby(['AM_LEGAL', variavel]).size().reset_index()
        b = pd.concat([a_[a_['AM_LEGAL'] == 1].reset_index(), a[['Regiao',variavel]].groupby(['Regiao', variavel]).size().reset_index()], axis=0)
        b = b.drop(columns=['index', 'AM_LEGAL', variavel]).fillna('AM_LEGAL')[['Regiao', 0]]
    
        df_base = pd.merge(df_base, b, on='Regiao', how='left', suffixes=(None, '_y'))
    
    variaveis.insert(0, 'Regiao')
    df_base.columns = variaveis
    df_base_ = df_base.copy().T
    df_base_.columns = ['AM_LEGAL','1 - Norte','2 - Nordeste','3 - Sudeste','4 - Sul','5 - Centro-Oeste']
    df_base_.drop(index=df_base_.index[0], axis=0, inplace=True)
    # Calculo do valor percentual
    df_base_['AM_LEGAL (%)'] = df_base_['AM_LEGAL'] / 775 * 100
    df_base_['1 - N (%)'] = df_base_['1 - Norte'] / 450 * 100
    df_base_['2 - NE (%)'] = df_base_['2 - Nordeste'] / 1794 * 100
    df_base_['3 - SE (%)'] = df_base_['3 - Sudeste'] / 1668 * 100
    df_base_['4 - S (%)'] = df_base_['4 - Sul'] / 1191 * 100
    df_base_['5 - CO (%)'] = df_base_['5 - Centro-Oeste'] / 467 * 100
    # Calculo da média ponderada entre as regiões do Brasil - Exclui Amazônia Legal
    df_base_['Media Pond'] = ((df_base_['1 - N (%)']*450 +
                                              df_base_['2 - NE (%)']*1794 +
                                              df_base_['3 - SE (%)']*1668 +
                                              df_base_['4 - S (%)']*1198 +
                                              df_base_['5 - CO (%)']*467)/(5570)
                                             )
    df_base_ = df_base_[['AM_LEGAL', 'AM_LEGAL (%)', 'Media Pond',
                        '1 - Norte','1 - N (%)',
                        '2 - Nordeste','2 - NE (%)',
                        '3 - Sudeste','3 - SE (%)',
                        '4 - Sul','4 - S (%)',
                        '5 - Centro-Oeste', '5 - CO (%)']]
    df_base_ = df_base_.fillna(0)
    return df_base_

## 0.2. IMPORTS DATA

### 0.2.0 ABA - AMAZONIA LEGAL

In [5]:
# IBGE municipios AM Legal
am_legal = pd.read_excel('../data/MUNIC/lista_de_municipios_Amazonia_Legal_2020.xls', sheet_name='Planilha1')
am_legal.drop(columns=['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], inplace=True)

# Aba com municipios
municipios_brasil = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Variáveis externa')

am_legal = am_legal[['CD_MUN']]
am_legal['AM_LEGAL'] = 1

df_amz_legal = pd.merge(municipios_brasil, am_legal, left_on='CodMun', right_on='CD_MUN', how='left')
df_amz_legal = df_amz_legal.fillna(0)
df_amz_legal = df_amz_legal.drop(columns=['Regiao', 'UF', 'Cod UF', 'Mun', 'Populacao', 'Faixa_pop', 'CD_MUN'])
df_amz_legal.head()

,CodMun,AM_LEGAL
0,1100015,1.00
1,1100023,1.00
2,1100031,1.00
3,1100049,1.00
4,1100056,1.00


### 0.2.1 ABA - RECURSOS HUMANOS
- Analisar a composição da administração pública

In [6]:
# Leitura dos dados da base fornecida
dfr_munic_rh = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Recursos humanos')
# renomeando para melhor entendimento
map_df = {'Mreh0111': '1_1_rh_qd_pess_dir_estatuarios',
          'Mreh0112': '1_1_rh_qd_pess_dir_celetistas',
          'Mreh0113': '1_1_rh_qd_pess_dir_so_comissionados',
          'Mreh0114': '1_1_rh_qd_pess_dir_estagiarios',
          'Mreh0115': '1_1_rh_qd_pess_dir_sem_vinculo',
          'Mreh0116': '1_1_rh_qd_pess_dir_total',
          'Mreh02'  : '1_2_rh_qd_pess_ind_adm_ind_exist',
          'Mreh0311': '1_2_rh_qd_pess_ind_estatuarios',
          'Mreh0312': '1_2_rh_qd_pess_ind_celetistas',
          'Mreh0313': '1_2_rh_qd_pess_ind_so_comissionados',
          'Mreh0314': '1_2_rh_qd_pess_ind_estagiarios',
          'Mreh0315': '1_2_rh_qd_pess_ind_sem_vinculo',
          'Mreh0316': '1_2_rh_qd_pess_ind_total'}
dfr_munic_rh = dfr_munic_rh.rename(columns=map_df)
dfr_munic_rh = pd.merge(dfr_munic_rh, df_amz_legal, on='CodMun', how='left')

In [7]:
#dfr_munic_rh.to_csv('../dados/munic/2020_munic_recursos_humanos.csv', index=False)

In [8]:
# ajustando nome dos campos para utilização da função agg
map_str = {'Recusa': 0, 'Não informou': 0, '(*) Não soube informar':0, '-':0}
dfr_munic_rh = dfr_munic_rh.replace({'1_1_rh_qd_pess_dir_total': map_str})
dfr_munic_rh = dfr_munic_rh.replace({'1_2_rh_qd_pess_ind_total': map_str})

In [9]:
dfr_munic_rh.head()

,CodMun,UF,Cod UF,Mun,Faixa_pop,Regiao,1_1_rh_qd_pess_dir_estatuarios,1_1_rh_qd_pess_dir_celetistas,1_1_rh_qd_pess_dir_so_comissionados,1_1_rh_qd_pess_dir_estagiarios,1_1_rh_qd_pess_dir_sem_vinculo,1_1_rh_qd_pess_dir_total,1_2_rh_qd_pess_ind_adm_ind_exist,1_2_rh_qd_pess_ind_estatuarios,1_2_rh_qd_pess_ind_celetistas,1_2_rh_qd_pess_ind_so_comissionados,1_2_rh_qd_pess_ind_estagiarios,1_2_rh_qd_pess_ind_sem_vinculo,1_2_rh_qd_pess_ind_total,AM_LEGAL
0,1100015,RO,11,Alta Floresta DOeste,4 - 20001 até 50000,1 - Norte,899,19,47,0,0,965,Sim,0,12,2,0,0,14,1.00
1,1100023,RO,11,Ariquemes,6 - 100001 até 500000,1 - Norte,2260,194,213,76,54,2797,Sim,6,0,4,0,0,10,1.00
2,1100031,RO,11,Cabixi,2 - 5001 até 10000,1 - Norte,271,24,54,0,14,363,Não,-,-,-,-,-,0,1.00
3,1100049,RO,11,Cacoal,5 - 50001 até 100000,1 - Norte,1698,10,222,0,27,1957,Sim,99,0,44,0,0,143,1.00
4,1100056,RO,11,Cerejeiras,3 - 10001 até 20000,1 - Norte,530,35,53,1,31,650,Não,-,-,-,-,-,0,1.00


In [10]:
# analisando a quantidade de pessoas alocadas na administração pública

display(dfr_munic_rh[['AM_LEGAL', '1_1_rh_qd_pess_dir_total', '1_2_rh_qd_pess_ind_total']].groupby('AM_LEGAL').agg({'1_1_rh_qd_pess_dir_total':['count', 'mean'],
                                                                                                        '1_2_rh_qd_pess_ind_total':['count', 'mean']}))
display(dfr_munic_rh[['Regiao', '1_1_rh_qd_pess_dir_total', '1_2_rh_qd_pess_ind_total']].groupby('Regiao').agg({'1_1_rh_qd_pess_dir_total':['count', 'mean'],
                                                                                                        '1_2_rh_qd_pess_ind_total':['mean']}))

1_1_rh_qd_pess_dir_total         1_2_rh_qd_pess_ind_total      
                            count    mean                    count  mean
AM_LEGAL                                                                
0.00                         4798 1091.19                     4798 53.61
1.00                          772 1160.28                      772 36.76

1_1_rh_qd_pess_dir_total         1_2_rh_qd_pess_ind_total
                                    count    mean                     mean
Regiao                                                                    
1 - Norte                             450 1267.61                    17.04
2 - Nordeste                         1794 1111.03                    41.80
3 - Sudeste                          1668 1319.32                    78.29
4 - Sul                              1191  703.46                    35.32
5 - Centro-Oeste                      467 1133.18                    64.85

**Conclusões:** de maneira geral a quantidade média de funcionários lotados nas prefeituras são parecidos com exceção do sul do país com quase metade do valor.

### 0.2.2 ABA - HABITAÇÃO

In [11]:
# Leitura dos dados da base fornecida
dfr_munic_hab = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Habitação')
map_df = {'Mhab01'  : '2_1_hab_orgao_gestor_caract',
          'Mhab03'  : '2_1_hab_orgao_gestor_sexo_tit',
          'Mhab04'  : '2_1_hab_orgao_gestor_idade_tit',
          'Mhab05'  : '2_1_hab_orgao_gestor_cor_tit',
          'Mhab051' : '2_1_hab_orgao_gestor_resp_proprio_tit',
          'Mhab06'  : '2_1_hab_orgao_gestor_escola_tit',
          'Mhab07'  : '2_1_hab_plan_municp_hab_existencia',
          'Mhab071' : '2_1_hab_plan_municp_hab_arct_pde',
          'Mhab081' : '2_1_hab_plan_municp_hab_integ_acoes_c_demais_pol_urb_soc',
          'Mhab082' : '2_1_hab_plan_municp_hab_aprov_infraestrutura',
          'Mhab083' : '2_1_hab_plan_municp_hab_prov_urb_recup_favela_reg_fund',
          'Mhab084' : '2_1_hab_plan_municp_hab_garantir_remocao_area_de_risco',
          'Mhab085' : '2_1_hab_plan_municp_hab_coibir_nv_ocup_inadeq',
          'Mhab086' : '2_1_hab_plan_municp_hab_prod_nv_und_his',
          'Mhab087' : '2_1_hab_plan_municp_hab_otim_rec_fed_est_mun',
          'Mhab088' : '2_1_hab_plan_municp_hab_prior_area_risco',
          'Mhab089' : '2_1_hab_plan_municp_hab_outros',
          'Mhab09'  : '2_1_hab_plan_municp_hab_conf_hab_4anos',
          'Mhab091' : '2_1_hab_plan_municp_hab_conf_hab_ref_pde',
          'Mhab10'  : '2_2_hab_consel_munic_hab_existencia',
          'Mhab101b': '2_2_hab_consel_munic_hab_ano_cria',
          'Mhab102' : '2_2_hab_consel_munic_hab_formacao',
          'Mhab111' : '2_2_hab_consel_munic_hab_conselho_consultivo',
          'Mhab112' : '2_2_hab_consel_munic_hab_conselho_deliberativo',
          'Mhab113' : '2_2_hab_consel_munic_hab_conselho_normativo',
          'Mhab114' : '2_2_hab_consel_munic_hab_conselho_fiscalizador',
          'Mhab12'  : '2_2_hab_consel_munic_hab_qtd_reuniao_12meses',
          'Mhab13'  : '2_2_hab_consel_munic_hab_numero_conselheiros',
          'Mhab141' : '2_2_hab_consel_munic_hab_cap_membros_period',
          'Mhab142' : '2_2_hab_consel_munic_hab_cap_membros_ocasin',
          'Mhab143' : '2_2_hab_consel_munic_hab_cap_membros_nrealiza',
          'Mhab15'  : '2_2_hab_consel_munic_hab_infra_exist',
          'Mhab1511': '2_2_hab_consel_munic_hab_infra_sala',
          'Mhab1512': '2_2_hab_consel_munic_hab_infra_compu',
          'Mhab1513': '2_2_hab_consel_munic_hab_infra_impres',
          'Mhab1514': '2_2_hab_consel_munic_hab_infra_intern',
          'Mhab1515': '2_2_hab_consel_munic_hab_infra_veicu',
          'Mhab1516': '2_2_hab_consel_munic_hab_infra_telef',
          'Mhab1517': '2_2_hab_consel_munic_hab_infra_diarias',
          'Mhab1518': '2_2_hab_consel_munic_hab_infra_dot_orc_propria',
          'Mhab16'  : '2_2_hab_fund_munic_hab_existencia',
          'Mhab161' : '2_2_hab_fund_munic_hab_gestor_pert_cons_m_hab',
          'Mhab17'  : '2_2_hab_fund_munic_hab_reune_recursos',
          'Mhab18'  : '2_3_hab_cad_prog_hab_existencia',
          'Mhab181' : '2_3_hab_cad_prog_hab_ano_realiz',
          'Mhab182' : '2_3_hab_cad_prog_hab_cadast_inform',
          'Mhab183' : '2_3_hab_cad_prog_hab_pretend_familia',
          'Mhab191' : '2_4_hab_hab_irreg_favelas',
          'Mhab192' : '2_4_hab_hab_irreg_corticos',
          'Mhab193' : '2_4_hab_hab_irreg_loteamento',
          'Mhab194' : '2_4_hab_hab_irreg_ocupa_terrenos',
          'Mhab195' : '2_4_hab_hab_irreg_nenhum',
          'Mhab201' : '2_5_hab_prog_hab_18_19_constr_hab',
          'Mhab202' : '2_5_hab_prog_hab_18_19_aquisicao_hab',
          'Mhab203' : '2_5_hab_prog_hab_18_19_melhoria_hab',
          'Mhab204' : '2_5_hab_prog_hab_18_19_oferta_mat_const',
          'Mhab205' : '2_5_hab_prog_hab_18_19_oferta_lotes',
          'Mhab206' : '2_5_hab_prog_hab_18_19_reg_fundiaria',
          'Mhab207' : '2_5_hab_prog_hab_18_19_urb_assent',
          'Mhab208' : '2_5_hab_prog_hab_18_19_nenhum',
          'Mhab21'  : '2_5_hab_prog_hab_18_19_aluguel_social'
         }
dfr_munic_hab = dfr_munic_hab.rename(columns=map_df)
dfr_munic_hab = pd.merge(dfr_munic_hab, df_amz_legal, on='CodMun', how='left')

In [12]:
#dfr_munic_hab.to_csv('../dados/munic/2020_munic_habitacao.csv', index=False)

In [13]:
dfr_munic_hab.head(1)

,CodMun,UF,Cod UF,Mun,Faixa_pop,Regiao,2_1_hab_orgao_gestor_caract,2_1_hab_orgao_gestor_sexo_tit,2_1_hab_orgao_gestor_idade_tit,2_1_hab_orgao_gestor_cor_tit,2_1_hab_orgao_gestor_resp_proprio_tit,2_1_hab_orgao_gestor_escola_tit,2_1_hab_plan_municp_hab_existencia,2_1_hab_plan_municp_hab_arct_pde,2_1_hab_plan_municp_hab_integ_acoes_c_demais_pol_urb_soc,2_1_hab_plan_municp_hab_aprov_infraestrutura,2_1_hab_plan_municp_hab_prov_urb_recup_favela_reg_fund,2_1_hab_plan_municp_hab_garantir_remocao_area_de_risco,2_1_hab_plan_municp_hab_coibir_nv_ocup_inadeq,2_1_hab_plan_municp_hab_prod_nv_und_his,2_1_hab_plan_municp_hab_otim_rec_fed_est_mun,2_1_hab_plan_municp_hab_prior_area_risco,2_1_hab_plan_municp_hab_outros,2_1_hab_plan_municp_hab_conf_hab_4anos,2_1_hab_plan_municp_hab_conf_hab_ref_pde,2_2_hab_consel_munic_hab_existencia,2_2_hab_consel_munic_hab_ano_cria,2_2_hab_consel_munic_hab_formacao,2_2_hab_consel_munic_hab_conselho_consultivo,2_2_hab_consel_munic_hab_conselho_deliberativo,2_2_hab_consel_munic_hab_conselho_normativo,2_2_hab_consel_munic_hab_conselho_fiscalizador,2_2_hab_consel_munic_hab_qtd_reuniao_12meses,2_2_hab_consel_munic_hab_numero_conselheiros,2_2_hab_consel_munic_hab_cap_membros_period,2_2_hab_consel_munic_hab_cap_membros_ocasin,2_2_hab_consel_munic_hab_cap_membros_nrealiza,2_2_hab_consel_munic_hab_infra_exist,2_2_hab_consel_munic_hab_infra_sala,2_2_hab_consel_munic_hab_infra_compu,2_2_hab_consel_munic_hab_infra_impres,2_2_hab_consel_munic_hab_infra_intern,2_2_hab_consel_munic_hab_infra_veicu,2_2_hab_consel_munic_hab_infra_telef,2_2_hab_consel_munic_hab_infra_diarias,2_2_hab_consel_munic_hab_infra_dot_orc_propria,2_2_hab_fund_munic_hab_existencia,2_2_hab_fund_munic_hab_gestor_pert_cons_m_hab,2_2_hab_fund_munic_hab_reune_recursos,2_3_hab_cad_prog_hab_existencia,2_3_hab_cad_prog_hab_ano_realiz,2_3_hab_cad_prog_hab_cadast_inform,2_3_hab_cad_prog_hab_pretend_familia,2_4_hab_hab_irreg_favelas,2_4_hab_hab_irreg_corticos,2_4_hab_hab_irreg_loteamento,2_4_hab_hab_irreg_ocupa_terrenos,2_4_hab_hab_irreg_nenhum,2_5_hab_prog_hab_18_19_constr_hab,2_5_hab_prog_hab_18_19_aquisicao_hab,2_5_hab_prog_hab_18_19_melhoria_hab,2_5_hab_prog_hab_18_19_oferta_mat_const,2_5_hab_prog_hab_18_19_oferta_lotes,2_5_hab_prog_hab_18_19_reg_fundiaria,2_5_hab_prog_hab_18_19_urb_assent,2_5_hab_prog_hab_18_19_nenhum,2_5_hab_prog_hab_18_19_aluguel_social,AM_LEGAL
0,1100015,RO,11,Alta Floresta DOeste,4 - 20001 até 50000,1 - Norte,Não possui estrutura,-,-,-,-,-,Sim,Não,Sim,Sim,Sim,Sim,Não,Sim,Sim,Não,Não,Não,-,Sim,2008,Paritário,Sim,Sim,Não,Sim,0,11,Não,Não,Sim,Não,-,-,-,-,-,-,-,-,Sim,Sim,Não,Sim,2019,Não,Não,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Sim,Sim,1.00


#### análise programa habitacional _ cadastro familia
CONCLUSÃO: Em termos proporcionais a divisão da existência de programa habitacional é parecido na região da AM Legal em relação ao restante do país. Vale ressaltar que uma secretaria exclusiva para este assunto é superior a média nacional na AM Legal

In [14]:
fam = dfr_munic_hab[dfr_munic_hab['2_3_hab_cad_prog_hab_existencia'] == 'Sim']
df_fam = fam[['AM_LEGAL','2_1_hab_orgao_gestor_caract', '2_3_hab_cad_prog_hab_existencia']].groupby(['AM_LEGAL','2_1_hab_orgao_gestor_caract', '2_3_hab_cad_prog_hab_existencia']).size().reset_index()

df_am = df_fam[df_fam['AM_LEGAL'] == 1][['2_1_hab_orgao_gestor_caract', 0]]
df_nam = df_fam[df_fam['AM_LEGAL'] == 0][['2_1_hab_orgao_gestor_caract', 0]]
df_fam = df_fam[['2_1_hab_orgao_gestor_caract', '2_3_hab_cad_prog_hab_existencia']].drop_duplicates().reset_index()
df_fam = pd.merge(df_fam, df_am, on='2_1_hab_orgao_gestor_caract', how='left').rename(columns={0:'AM_LEGAL'})
df_fam['AM_LEGAL_PCT'] = df_fam['AM_LEGAL'] / dfr_munic_hab['AM_LEGAL'].sum() *100
df_fam = pd.merge(df_fam, df_nam, on='2_1_hab_orgao_gestor_caract', how='left').rename(columns={0:'RESTO_PAIS'})
df_fam['RESTO_PAIS_PCT'] = df_fam['RESTO_PAIS'] / (len(dfr_munic_hab) - dfr_munic_hab['AM_LEGAL'].sum()) *100
df_fam

,index,2_1_hab_orgao_gestor_caract,2_3_hab_cad_prog_hab_existencia,AM_LEGAL,AM_LEGAL_PCT,RESTO_PAIS,RESTO_PAIS_PCT
0,0,Não possui estrutura,Sim,123,15.93,861,17.94
1,1,Secretaria em conjunto com outras políticas se...,Sim,94,12.18,615,12.82
2,2,Secretaria exclusiva,Sim,32,4.15,165,3.44
3,3,Setor subordinado a outra secretaria,Sim,220,28.50,1374,28.64
4,4,Setor subordinado diretamente à chefia do Exec...,Sim,13,1.68,141,2.94
5,5,Órgão da administração indireta,Sim,4,0.52,25,0.52


#### Habitação - Análise Habitação Irregular
CONCLUSÃO:  De maneira geral, a existência de favelas é um dos principais problemas de habitação na região da AM Legal, no qual grande contributo vem da região norte do país.

In [15]:
variaveis = ['2_4_hab_hab_irreg_favelas', '2_4_hab_hab_irreg_corticos', '2_4_hab_hab_irreg_loteamento', '2_4_hab_hab_irreg_ocupa_terrenos', '2_4_hab_hab_irreg_nenhum']
analise_brasil(dfr_munic_hab, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
2_4_hab_hab_irreg_favelas,163,21.03,18.86,131,29.11,376,20.96,267,16.01,241,20.24,34,7.28
2_4_hab_hab_irreg_corticos,125,16.13,15.54,83,18.44,306,17.06,313,18.76,109,9.15,54,11.56
2_4_hab_hab_irreg_loteamento,493,63.61,66.68,312,69.33,1030,57.41,1212,72.66,857,71.96,298,63.81
2_4_hab_hab_irreg_ocupa_terrenos,126,16.26,15.44,81,18.00,349,19.45,195,11.69,157,13.18,77,16.49
2_4_hab_hab_irreg_nenhum,159,20.52,26.37,75,16.67,564,31.44,395,23.68,289,24.27,144,30.84


#### Habitação - Programas Sociais
CONCLUSÃO: 
- Existe uma boa capilaridade de programas sociais na AM Legal comparado a média ponderada nacional relacionado à questão de terra.
- Por outro lado, isso não acontece para fornecer moradias prontas para as pessoas que moram na região.

In [16]:
variaveis = ['2_5_hab_prog_hab_18_19_constr_hab','2_5_hab_prog_hab_18_19_aquisicao_hab','2_5_hab_prog_hab_18_19_melhoria_hab',
'2_5_hab_prog_hab_18_19_oferta_mat_const','2_5_hab_prog_hab_18_19_oferta_lotes','2_5_hab_prog_hab_18_19_reg_fundiaria',
'2_5_hab_prog_hab_18_19_urb_assent','2_5_hab_prog_hab_18_19_nenhum','2_5_hab_prog_hab_18_19_aluguel_social']
analise_brasil(dfr_munic_hab, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
2_5_hab_prog_hab_18_19_constr_hab,161,20.77,25.28,124,27.56,396,22.07,392,23.50,359,30.14,135,28.91
2_5_hab_prog_hab_18_19_aquisicao_hab,39,5.03,6.06,19,4.22,120,6.69,98,5.88,59,4.95,41,8.78
2_5_hab_prog_hab_18_19_melhoria_hab,113,14.58,25.32,76,16.89,446,24.86,367,22.00,424,35.60,95,20.34
2_5_hab_prog_hab_18_19_oferta_mat_const,91,11.74,22.09,67,14.89,293,16.33,380,22.78,402,33.75,86,18.42
2_5_hab_prog_hab_18_19_oferta_lotes,141,18.19,13.14,112,24.89,175,9.75,163,9.77,161,13.52,120,25.70
2_5_hab_prog_hab_18_19_reg_fundiaria,238,30.71,30.62,128,28.44,306,17.06,607,36.39,452,37.95,210,44.97
2_5_hab_prog_hab_18_19_urb_assent,80,10.32,7.89,48,10.67,153,8.53,119,7.13,80,6.72,39,8.35
2_5_hab_prog_hab_18_19_nenhum,244,31.48,33.09,133,29.56,746,41.58,517,31.00,321,26.95,124,26.55
2_5_hab_prog_hab_18_19_aluguel_social,256,33.03,55.22,175,38.89,1042,58.08,1085,65.05,606,50.88,164,35.12


### 0.2.3 ABA - TRANSPORTE

In [17]:
# Leitura dos dados da base fornecida
dfr_munic_transp = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Transporte')
map_df = {'Mtra01'  : "3_1_transp_plan_munic_caract_orgao",
          'Mtra03'  : "3_1_transp_plan_munic_sexo_tit",
          'Mtra04'  : "3_1_transp_plan_munic_idade_tit",
          'Mtra05'  : "3_1_transp_plan_munic_raca_tit",
          'Mtra051' : "3_1_transp_plan_munic_resp_prop_tit",
          'Mtra06'  : "3_1_transp_plan_munic_escolar_tit",
          'Mtra07'  : "3_1_transp_plan_munic_existencia",
          'Mtra081' : "3_1_transp_plan_munic_plt_circ_viaria",
          'Mtra082' : "3_1_transp_plan_munic_organ_sistema",
          'Mtra083' : "3_1_transp_plan_munic_politic_tarifaria",
          'Mtra084' : "3_1_transp_plan_munic_normas_sistema",
          'Mtra085' : "3_1_transp_plan_munic_uso_ped_ciclista",
          'Mtra086' : "3_1_transp_plan_munic_uso_deficiente",
          'Mtra087' : "3_1_transp_plan_munic_outros",
          'Mtra09'  : "3_1_transp_plan_munic_conf_transporte4anos",
          'Mtra091' : "3_1_transp_plan_munic_considerado_plan_mun",
          'Mtra10'  : "3_2_transp_cons_munic_existencia",
          'Mtra101b': "3_2_transp_cons_munic_ano_cria",
          'Mtra102' : "3_2_transp_cons_munic_formacao",
          'Mtra111' : "3_2_transp_cons_munic_conselho_consultivo",
          'Mtra112' : "3_2_transp_cons_munic_conselho_deliberativo",
          'Mtra113' : "3_2_transp_cons_munic_conselho_normativo",
          'Mtra114' : "3_2_transp_cons_munic_conselho_fiscalizador",
          'Mtra12'  : "3_2_transp_cons_munic_qtd_reuniao_12meses",
          'Mtra13'  : "3_2_transp_cons_munic_numero_conselheiros",
          'Mtra141' : "3_2_transp_cons_munic_cap_membros_period",
          'Mtra142' : "3_2_transp_cons_munic_cap_membros_ocasin",
          'Mtra143' : "3_2_transp_cons_munic_cap_membros_nrealiza",
          'Mtra15'  : "3_2_transp_cons_munic_infra_exist",
          'Mtra1511': "3_2_transp_cons_munic_infra_sala",
          'Mtra1512': "3_2_transp_cons_munic_infra_compu",
          'Mtra1513': "3_2_transp_cons_munic_infra_impres",
          'Mtra1514': "3_2_transp_cons_munic_infra_intern",
          'Mtra1515': "3_2_transp_cons_munic_infra_veicu",
          'Mtra1516': "3_2_transp_cons_munic_infra_telef",
          'Mtra1517': "3_2_transp_cons_munic_infra_diarias",
          'Mtra1518': "3_2_transp_cons_munic_infra_dot_orc_propria",
          'Mtra16'  : "3_2_transp_fund_munic_existencia",
          'Mtra161' : "3_2_transp_fund_munic_gestor_pert_cons_m_trans",
          'Mtra17'  : "3_2_transp_fund_munic_financiado_acoes",
          'Mtra181' : "3_3_transp_exist_barco",
          'Mtra182' : "3_3_transp_exist_metro",
          'Mtra183' : "3_3_transp_exist_mototaxi",
          'Mtra184' : "3_3_transp_exist_taxi",
          'Mtra185' : "3_3_transp_exist_trem",
          'Mtra186' : "3_3_transp_exist_van",
          'Mtra187' : "3_3_transp_exist_aviao",
          'Mtra188' : "3_3_transp_exist_app",
          'Mtra189' : "3_3_transp_exist_nenhum",
          'Mtra19'  : "3_3_transp_onibus_existencia",
          'Mtra1911': "3_3_transp_onibus_concessao",
          'Mtra192' : "3_3_transp_onibus_concessao_licit",
          'Mtra1912': "3_3_transp_onibus_permissao",
          'Mtra193' : "3_3_transp_onibus_permissao_licit",
          'Mtra1913': "3_3_transp_onibus_autorizacao",
          'Mtra1914': "3_3_transp_onibus_serv_by_pref",
          'Mtra1915': "3_3_transp_onibus_nao_regulament",
          'Mtra201' : "3_3_transp_onibus_isencao_idoso",
          'Mtra202' : "3_3_transp_onibus_isencao_est_publica",
          'Mtra203' : "3_3_transp_onibus_isencao_est_partic",
          'Mtra204' : "3_3_transp_onibus_isencao_carteiros",
          'Mtra205' : "3_3_transp_onibus_isencao_pcd",
          'Mtra206' : "3_3_transp_onibus_isencao_policiais",
          'Mtra207' : "3_3_transp_onibus_isencao_professor",
          'Mtra208' : "3_3_transp_onibus_isencao_crianca_5anos",
          'Mtra209' : "3_3_transp_onibus_isencao_outros",
          'Mtra2010': "3_3_transp_onibus_isencao_toda_populacao",
          'Mtra2011': "3_3_transp_onibus_isencao_nenhum_passag",
          'Mtra21'  : "3_3_transp_onibus_pcd_existencia",
          'Mtra221' : "3_3_transp_onibus_pcd_piso_baixo",
          'Mtra222' : "3_3_transp_onibus_pcd_plataforma",
          'Mtra223' : "3_3_transp_onibus_pcd_elevatoria",
          'Mtra224' : "3_3_transp_onibus_pcd_nao_sabe",
          'Mtra23'  : "3_3_transp_intermunic_existencia",
          'Mtra231' : "3_3_transp_intermunic_atende_municip",
          'Mtra24'  : "3_3_transp_ciclovia",
          'Mtra25'  : "3_3_transp_bicicletario"
         }
dfr_munic_transp = dfr_munic_transp.rename(columns=map_df)
dfr_munic_transp = pd.merge(dfr_munic_transp, df_amz_legal, on='CodMun', how='left')

In [18]:
#dfr_munic_transp.to_csv('../dados/munic/2020_munic_transporte.csv', index=False)

In [19]:
dfr_munic_transp.head(1)

,CodMun,UF,Cod UF,Mun,Faixa_pop,Regiao,3_1_transp_plan_munic_caract_orgao,3_1_transp_plan_munic_sexo_tit,3_1_transp_plan_munic_idade_tit,3_1_transp_plan_munic_raca_tit,3_1_transp_plan_munic_resp_prop_tit,3_1_transp_plan_munic_escolar_tit,3_1_transp_plan_munic_existencia,3_1_transp_plan_munic_plt_circ_viaria,3_1_transp_plan_munic_organ_sistema,3_1_transp_plan_munic_politic_tarifaria,3_1_transp_plan_munic_normas_sistema,3_1_transp_plan_munic_uso_ped_ciclista,3_1_transp_plan_munic_uso_deficiente,3_1_transp_plan_munic_outros,3_1_transp_plan_munic_conf_transporte4anos,3_1_transp_plan_munic_considerado_plan_mun,3_2_transp_cons_munic_existencia,3_2_transp_cons_munic_ano_cria,3_2_transp_cons_munic_formacao,3_2_transp_cons_munic_conselho_consultivo,3_2_transp_cons_munic_conselho_deliberativo,3_2_transp_cons_munic_conselho_normativo,3_2_transp_cons_munic_conselho_fiscalizador,3_2_transp_cons_munic_qtd_reuniao_12meses,3_2_transp_cons_munic_numero_conselheiros,3_2_transp_cons_munic_cap_membros_period,3_2_transp_cons_munic_cap_membros_ocasin,3_2_transp_cons_munic_cap_membros_nrealiza,3_2_transp_cons_munic_infra_exist,3_2_transp_cons_munic_infra_sala,3_2_transp_cons_munic_infra_compu,3_2_transp_cons_munic_infra_impres,3_2_transp_cons_munic_infra_intern,3_2_transp_cons_munic_infra_veicu,3_2_transp_cons_munic_infra_telef,3_2_transp_cons_munic_infra_diarias,3_2_transp_cons_munic_infra_dot_orc_propria,3_2_transp_fund_munic_existencia,3_2_transp_fund_munic_gestor_pert_cons_m_trans,3_2_transp_fund_munic_financiado_acoes,3_3_transp_exist_barco,3_3_transp_exist_metro,3_3_transp_exist_mototaxi,3_3_transp_exist_taxi,3_3_transp_exist_trem,3_3_transp_exist_van,3_3_transp_exist_aviao,3_3_transp_exist_app,3_3_transp_exist_nenhum,3_3_transp_onibus_existencia,3_3_transp_onibus_concessao,3_3_transp_onibus_concessao_licit,3_3_transp_onibus_permissao,3_3_transp_onibus_permissao_licit,3_3_transp_onibus_autorizacao,3_3_transp_onibus_serv_by_pref,3_3_transp_onibus_nao_regulament,3_3_transp_onibus_isencao_idoso,3_3_transp_onibus_isencao_est_publica,3_3_transp_onibus_isencao_est_partic,3_3_transp_onibus_isencao_carteiros,3_3_transp_onibus_isencao_pcd,3_3_transp_onibus_isencao_policiais,3_3_transp_onibus_isencao_professor,3_3_transp_onibus_isencao_crianca_5anos,3_3_transp_onibus_isencao_outros,3_3_transp_onibus_isencao_toda_populacao,3_3_transp_onibus_isencao_nenhum_passag,3_3_transp_onibus_pcd_existencia,3_3_transp_onibus_pcd_piso_baixo,3_3_transp_onibus_pcd_plataforma,3_3_transp_onibus_pcd_elevatoria,3_3_transp_onibus_pcd_nao_sabe,3_3_transp_intermunic_existencia,3_3_transp_intermunic_atende_municip,3_3_transp_ciclovia,3_3_transp_bicicletario,AM_LEGAL
0,1100015,RO,11,Alta Floresta DOeste,4 - 20001 até 50000,1 - Norte,Não possui estrutura,-,-,-,-,-,Não,-,-,-,-,-,-,-,Não,-,Não,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Não,-,-,Não,Não,Sim,Sim,Não,Não,Não,Sim,Não,Não,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Não,-,Não,Não,1.00


#### Transportes - Modais existentes
- Barco, mototaxi, van e avião são os principais de deslocamento na AM Legal.

In [20]:
variaveis = ['3_3_transp_exist_barco','3_3_transp_exist_metro','3_3_transp_exist_mototaxi','3_3_transp_exist_taxi',
'3_3_transp_exist_trem','3_3_transp_exist_van','3_3_transp_exist_aviao','3_3_transp_exist_app','3_3_transp_exist_nenhum']
analise_brasil(dfr_munic_transp, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
3_3_transp_exist_barco,216.00,27.87,8.25,182.00,40.44,148.00,8.25,67.00,4.02,43.00,3.61,19.00,4.07
3_3_transp_exist_metro,0.00,0.00,0.00,0.00,0.00,11.00,0.61,4.00,0.24,4.00,0.34,1.00,0.21
3_3_transp_exist_mototaxi,531.00,68.52,49.52,329.00,73.11,1411.00,78.65,529.00,31.71,200.00,16.79,288.00,61.67
3_3_transp_exist_taxi,515.00,66.45,79.07,286.00,63.56,1239.00,69.06,1488.00,89.21,1023.00,85.89,362.00,77.52
3_3_transp_exist_trem,4.00,0.52,0.00,1.00,0.22,21.00,1.17,65.00,3.90,4.00,0.34,0.00,0.00
3_3_transp_exist_van,436.00,56.26,59.76,248.00,55.11,1485.00,82.78,897.00,53.78,485.00,40.72,211.00,45.18
3_3_transp_exist_aviao,72.00,9.29,2.93,56.00,12.44,24.00,1.34,34.00,2.04,25.00,2.10,24.00,5.14
3_3_transp_exist_app,85.00,10.97,15.79,48.00,10.67,98.00,5.46,399.00,23.92,251.00,21.07,82.00,17.56
3_3_transp_exist_nenhum,33.00,4.26,7.09,12.00,2.67,70.00,3.90,121.00,7.25,121.00,10.16,70.00,14.99


#### Transportes - Ônibus
CONCLUSÃO: 
- Há uma baixa permeabilidade de utilização de ônibus intermunicipalmente e também entre municípios na AM Legal.

In [21]:
variaveis = ['3_3_transp_onibus_existencia','3_3_transp_onibus_isencao_idoso','3_3_transp_onibus_isencao_est_publica','3_3_transp_onibus_isencao_est_partic','3_3_transp_onibus_isencao_carteiros',
'3_3_transp_onibus_isencao_pcd','3_3_transp_onibus_isencao_policiais','3_3_transp_onibus_isencao_professor','3_3_transp_onibus_isencao_crianca_5anos',
'3_3_transp_onibus_isencao_outros','3_3_transp_onibus_isencao_toda_populacao','3_3_transp_onibus_isencao_nenhum_passag',]
analise_brasil(dfr_munic_transp, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
3_3_transp_onibus_existencia,137,17.68,31.05,85,18.89,354,19.73,739,44.30,442,37.11,107,22.91
3_3_transp_onibus_isencao_idoso,95,12.26,20.59,62,13.78,176,9.81,528,31.65,311,26.11,68,14.56
3_3_transp_onibus_isencao_est_publica,30,3.87,8.04,17,3.78,54,3.01,209,12.53,141,11.84,26,5.57
3_3_transp_onibus_isencao_est_partic,9,1.16,2.64,5,1.11,16,0.89,61,3.66,47,3.95,18,3.85
3_3_transp_onibus_isencao_carteiros,15,1.94,3.81,11,2.44,28,1.56,111,6.65,52,4.37,10,2.14
3_3_transp_onibus_isencao_pcd,56,7.23,12.87,38,8.44,105,5.85,352,21.10,180,15.11,41,8.78
3_3_transp_onibus_isencao_policiais,54,6.97,9.15,42,9.33,87,4.85,239,14.33,112,9.40,29,6.21
3_3_transp_onibus_isencao_professor,8,1.03,1.42,6,1.33,12,0.67,32,1.92,27,2.27,2,0.43
3_3_transp_onibus_isencao_crianca_5anos,78,10.06,14.51,54,12.00,129,7.19,366,21.94,208,17.46,50,10.71
3_3_transp_onibus_isencao_outros,8,1.03,2.00,6,1.33,23,1.28,51,3.06,24,2.02,7,1.50


In [22]:
variaveis = ['3_3_transp_intermunic_existencia','3_3_transp_intermunic_atende_municip',]
analise_brasil(dfr_munic_transp, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
3_3_transp_intermunic_existencia,468,60.39,81.28,302,67.11,1277,71.18,1578,94.60,1024,85.98,340,72.81
3_3_transp_intermunic_atende_municip,213,27.48,35.10,156,34.67,444,24.75,727,43.59,511,42.91,114,24.41


#### Bicicletas
CONCLUSÃO: 
- Comparado a média nacional, a AM Legal possui a menor permeabilidade para promover esse tipo de transporte

In [23]:
variaveis = ['3_3_transp_ciclovia','3_3_transp_bicicletario']
analise_brasil(dfr_munic_transp, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
3_3_transp_ciclovia,89,11.48,20.92,47,10.44,227,12.65,423,25.36,372,31.23,94,20.13
3_3_transp_bicicletario,25,3.23,6.85,18,4.00,48,2.68,177,10.61,109,9.15,29,6.21


### 0.2.4 ABA - AGROPECUARIO

In [24]:
# Leitura dos dados da base fornecida
dfr_munic_agropec = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Agropecuário')
map_df = {'Magr01'   : "4_1_agrop_gest_polit_caract_orgao",
          'Magr03'   : "4_1_agrop_gest_polit_sexo_tit",
          'Magr04'   : "4_1_agrop_gest_polit_idade_tit",
          'Magr05'   : "4_1_agrop_gest_polit_raca_tit",
          'Magr051'  : "4_1_agrop_gest_polit_resp_prop_tit",
          'Magr06'   : "4_1_agrop_gest_polit_escolar_tit",
          'Magr07'   : "4_2_agrop_cons_desen_rural_existencia",
          'Magr071'  : "4_2_agrop_cons_desen_rural_ano_cria",
          'Magr072'  : "4_2_agrop_cons_desen_rural_formacao",
          'Magr081'  : "4_2_agrop_cons_desen_rural_conselho_consultivo",
          'Magr082'  : "4_2_agrop_cons_desen_rural_conselho_deliberativo",
          'Magr083'  : "4_2_agrop_cons_desen_rural_conselho_normativo",
          'Magr084'  : "4_2_agrop_cons_desen_rural_conselho_fiscalizador",
          'Magr09'   : "4_2_agrop_cons_desen_rural_qtd_reuniao_12meses",
          'Magr10'   : "4_2_agrop_cons_desen_rural_numero_conselheiros",
          'Magr111'  : "4_2_agrop_cons_desen_rural_cap_membros_period",
          'Magr112'  : "4_2_agrop_cons_desen_rural_cap_membros_ocasin",
          'Magr113'  : "4_2_agrop_cons_desen_rural_cap_membros_nrealiza",
          'Magr12'   : "4_2_agrop_cons_desen_rural_infra_exist",
          'Magr1211' : "4_2_agrop_cons_desen_rural_infra_sala",
          'Magr1212' : "4_2_agrop_cons_desen_rural_infra_compu",
          'Magr1213' : "4_2_agrop_cons_desen_rural_infra_impres",
          'Magr1214' : "4_2_agrop_cons_desen_rural_infra_intern",
          'Magr1215' : "4_2_agrop_cons_desen_rural_infra_veicu",
          'Magr1216' : "4_2_agrop_cons_desen_rural_infra_telef",
          'Magr1217' : "4_2_agrop_cons_desen_rural_infra_diarias",
          'Magr1218' : "4_2_agrop_cons_desen_rural_infra_dot_orc_propria",
          'Magr131'  : "4_3_agrop_prog_acoes_semetes_existencia",
          'Magr13111': "4_3_agrop_prog_acoes_semetes_dist_gratuita",
          'Magr13112': "4_3_agrop_prog_acoes_semetes_mais_barata",
          'Magr13113': "4_3_agrop_prog_acoes_semetes_financiado",
          'Magr13114': "4_3_agrop_prog_acoes_semetes_outros",
          'Magr132'  : "4_3_agrop_prog_acoes_mudas_existencia",
          'Magr13211': "4_3_agrop_prog_acoes_mudas_dist_gratuita",
          'Magr13212': "4_3_agrop_prog_acoes_mudas_mais_barata",
          'Magr13213': "4_3_agrop_prog_acoes_mudas_financiado",
          'Magr13214': "4_3_agrop_prog_acoes_mudas_outros",
          'Magr133'  : "4_3_agrop_prog_acoes_adubos_existencia",
          'Magr13311': "4_3_agrop_prog_acoes_adubos_dist_gratuita",
          'Magr13312': "4_3_agrop_prog_acoes_adubos_mais_barata",
          'Magr13313': "4_3_agrop_prog_acoes_adubos_financiado",
          'Magr13314': "4_3_agrop_prog_acoes_adubos_outros",
          'Magr134'  : "4_3_agrop_prog_acoes_racao_adub_existencia",
          'Magr13411': "4_3_agrop_prog_acoes_racao_adub_dist_gratuita",
          'Magr13412': "4_3_agrop_prog_acoes_racao_adub_mais_barata",
          'Magr13413': "4_3_agrop_prog_acoes_racao_adub_financiado",
          'Magr13414': "4_3_agrop_prog_acoes_racao_adub_outros",
          'Magr135'  : "4_3_agrop_prog_acoes_alevinos_existencia",
          'Magr13511': "4_3_agrop_prog_acoes_alevinos_dist_gratuita",
          'Magr13512': "4_3_agrop_prog_acoes_alevinos_mais_barata",
          'Magr13513': "4_3_agrop_prog_acoes_alevinos_financiado",
          'Magr13514': "4_3_agrop_prog_acoes_alevinos_outros",
          'Magr136'  : "4_3_agrop_prog_acoes_outros_insumos",
          'Magr141'  : "4_3_agrop_prog_acoes_maquin_cessao_temp_gratu",
          'Magr142'  : "4_3_agrop_prog_acoes_maquin_aluguel",
          'Magr143'  : "4_3_agrop_prog_acoes_maquin_outro",
          'Magr144'  : "4_3_agrop_prog_acoes_maquin_nenhumo",
          'Magr151'  : "4_3_agrop_prog_acoes_agric_organica",
          'Magr152'  : "4_3_agrop_prog_acoes_agric_familiar",
          'Magr153'  : "4_3_agrop_prog_acoes_aquicultura",
          'Magr154'  : "4_3_agrop_prog_acoes_pesca",
          'Magr155'  : "4_3_agrop_prog_acoes_hortas_comunitarias",
          'Magr16'   : "4_4_agrop_promo_festiv_existencia",
          'Magr171'  : "4_4_agrop_promo_festiv_festividades",
          'Magr172'  : "4_4_agrop_promo_festiv_premiacoes",
          'Magr18'   : "4_4_agrop_preven_prob_climatico",
          'Magr191'  : "4_5_agrop_assistec_municipal",
          'Magr192'  : "4_5_agrop_assistec_estadual",
          'Magr193'  : "4_5_agrop_assistec_federal",
          'Magr194'  : "4_5_agrop_assistec_naoexiste",
          'Magr20'   : "4_5_agrop_assistec_inst_conveniadas",
          'Magr211'  : "4_5_agrop_assistec_empres_privada",
          'Magr212'  : "4_5_agrop_assistec_ong",
          'Magr213'  : "4_5_agrop_assistec_outro",
          'Magr214'  : "4_5_agrop_assistec_nenhum",
          'Magr221'  : "4_5_agrop_prog_acoes_social_existencia",
          'Magr22111': "4_5_agrop_prog_acoes_social_educa",
          'Magr22112': "4_5_agrop_prog_acoes_social_saude",
          'Magr22113': "4_5_agrop_prog_acoes_social_alimen",
          'Magr22114': "4_5_agrop_prog_acoes_social_outros",
          'Magr222'  : "4_5_agrop_prog_acoes_artes_existencia",
          'Magr22211': "4_5_agrop_prog_acoes_artes_capacit",
          'Magr22212': "4_5_agrop_prog_acoes_artes_apoio_comer",
          'Magr22213': "4_5_agrop_prog_acoes_artes_associatv",
          'Magr22214': "4_5_agrop_prog_acoes_artes_outros",
          'Magr231'  : "4_6_agrop_associatv_cadpref_sind_rural",
          'Magr232'  : "4_6_agrop_associatv_cadpref_assoc_prod",
          'Magr233'  : "4_6_agrop_associatv_cadpref_cooperat",
          'Magr234'  : "4_6_agrop_associatv_cadpref_naopossui",
          'Magr241'  : "4_6_agrop_associatv_atua_munc_sind_rural",
          'Magr242'  : "4_6_agrop_associatv_atua_munc_assoc_prod",
          'Magr243'  : "4_6_agrop_associatv_atua_munc_cooperat",
          'Magr244'  : "4_6_agrop_associatv_atua_munc_outro",
          'Magr245'  : "4_6_agrop_associatv_atua_munc_nenhuma",
          'Magr251'  : "4_7_agrop_infra_centro_comercial",
          'Magr252'  : "4_7_agrop_infra_feiras",
          'Magr253'  : "4_7_agrop_infra_parque_exposicao",
          'Magr254'  : "4_7_agrop_infra_c_comercial_vegetal",
          'Magr255'  : "4_7_agrop_infra_outro",
          'Magr256'  : "4_7_agrop_infra_nenhum",
          'Magr261'  : "4_8_agrop_aq_prod_direto_prod",
          'Magr262'  : "4_8_agrop_aq_prod_atraves_entid",
          'Magr263'  : "4_8_agrop_aq_prod_prog_aquis",
          'Magr264'  : "4_8_agrop_aq_prod_outro",
          'Magr265'  : "4_8_agrop_aq_prod_nenhum",
          'Magr271'  : "4_8_agrop_vacina_gratuito",
          'Magr272'  : "4_8_agrop_vacina_maisbarato",
          'Magr273'  : "4_8_agrop_vacina_outro",
          'Magr274'  : "4_8_agrop_vacina_nenhum",
          'Magr28'   : "4_8_agrop_impl_sim",
          'Magr29'   : "4_8_agrop_abatedouro_existencia",
          'Magr291'  : "4_8_agrop_abatedouro_uso12meses",
          'Magr30'   : "4_8_agrop_exist_extracao_vegetal"
        }
dfr_munic_agropec = dfr_munic_agropec.rename(columns=map_df)
dfr_munic_agropec = pd.merge(dfr_munic_agropec, df_amz_legal, on='CodMun', how='left')

In [25]:
#dfr_munic_agropec.to_csv('../dados/munic/2020_munic_agropecuaria.csv', index=False)

In [26]:
dfr_munic_agropec[['AM_LEGAL','4_5_agrop_assistec_inst_conveniadas']].groupby(['AM_LEGAL', '4_5_agrop_assistec_inst_conveniadas']).size().reset_index()

,AM_LEGAL,4_5_agrop_assistec_inst_conveniadas,0
0,0.00,Instituições privadas,175
1,0.00,Instituições públicas,2802
2,0.00,Instituições públicas e privadas,835
3,0.00,Não informou,6
4,0.00,Não possui convênio,960
5,0.00,Recusa,20
6,1.00,Instituições privadas,13
7,1.00,Instituições públicas,392
8,1.00,Instituições públicas e privadas,89
9,1.00,Não informou,13


In [27]:
dfr_munic_agropec.columns.tolist()

['CodMun',
 'UF',
 'Cod UF',
 'Mun',
 'Faixa_pop',
 'Regiao',
 '4_1_agrop_gest_polit_caract_orgao',
 '4_1_agrop_gest_polit_sexo_tit',
 '4_1_agrop_gest_polit_idade_tit',
 '4_1_agrop_gest_polit_raca_tit',
 '4_1_agrop_gest_polit_resp_prop_tit',
 '4_1_agrop_gest_polit_escolar_tit',
 '4_2_agrop_cons_desen_rural_existencia',
 '4_2_agrop_cons_desen_rural_ano_cria',
 '4_2_agrop_cons_desen_rural_formacao',
 '4_2_agrop_cons_desen_rural_conselho_consultivo',
 '4_2_agrop_cons_desen_rural_conselho_deliberativo',
 '4_2_agrop_cons_desen_rural_conselho_normativo',
 '4_2_agrop_cons_desen_rural_conselho_fiscalizador',
 '4_2_agrop_cons_desen_rural_qtd_reuniao_12meses',
 '4_2_agrop_cons_desen_rural_numero_conselheiros',
 '4_2_agrop_cons_desen_rural_cap_membros_period',
 '4_2_agrop_cons_desen_rural_cap_membros_ocasin',
 '4_2_agrop_cons_desen_rural_cap_membros_nrealiza',
 '4_2_agrop_cons_desen_rural_infra_exist',
 '4_2_agrop_cons_desen_rural_infra_sala',
 '4_2_agrop_cons_desen_rural_infra_compu',
 '4_2_agro

#### Programa ou ação de acesso facilitado aos produtores
CONCLUSÃO: 
- A AM Legal possui um dos maiores programas de distribuição de sementes, adubos e mudas a nível nacional

In [28]:
variaveis = [#'4_3_agrop_prog_acoes_semetes_existencia',
'4_3_agrop_prog_acoes_semetes_dist_gratuita','4_3_agrop_prog_acoes_semetes_mais_barata','4_3_agrop_prog_acoes_semetes_financiado','4_3_agrop_prog_acoes_semetes_outros',
#'4_3_agrop_prog_acoes_mudas_existencia',
'4_3_agrop_prog_acoes_mudas_dist_gratuita','4_3_agrop_prog_acoes_mudas_mais_barata','4_3_agrop_prog_acoes_mudas_financiado','4_3_agrop_prog_acoes_mudas_outros',
#'4_3_agrop_prog_acoes_adubos_existencia',
'4_3_agrop_prog_acoes_adubos_dist_gratuita','4_3_agrop_prog_acoes_adubos_mais_barata','4_3_agrop_prog_acoes_adubos_financiado','4_3_agrop_prog_acoes_adubos_outros',
#'4_3_agrop_prog_acoes_racao_adub_existencia',
'4_3_agrop_prog_acoes_racao_adub_dist_gratuita','4_3_agrop_prog_acoes_racao_adub_mais_barata','4_3_agrop_prog_acoes_racao_adub_financiado','4_3_agrop_prog_acoes_racao_adub_outros',
#'4_3_agrop_prog_acoes_alevinos_existencia',
'4_3_agrop_prog_acoes_alevinos_dist_gratuita','4_3_agrop_prog_acoes_alevinos_mais_barata','4_3_agrop_prog_acoes_alevinos_financiado','4_3_agrop_prog_acoes_alevinos_outros',
'4_3_agrop_prog_acoes_outros_insumos',]
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_3_agrop_prog_acoes_semetes_dist_gratuita,368.00,47.48,33.02,195.00,43.33,1013.00,56.47,370.00,22.18,167.00,14.02,93.00,19.91
4_3_agrop_prog_acoes_semetes_mais_barata,15.00,1.94,8.83,9.00,2.00,72.00,4.01,128.00,7.67,267.00,22.42,14.00,3.00
4_3_agrop_prog_acoes_semetes_financiado,3.00,0.39,1.37,3.00,0.67,9.00,0.50,11.00,0.66,52.00,4.37,1.00,0.21
4_3_agrop_prog_acoes_semetes_outros,20.00,2.58,5.85,10.00,2.22,84.00,4.68,82.00,4.92,132.00,11.08,17.00,3.64
4_3_agrop_prog_acoes_mudas_dist_gratuita,366.00,47.23,40.21,248.00,55.11,859.00,47.88,595.00,35.67,331.00,27.79,205.00,43.90
4_3_agrop_prog_acoes_mudas_mais_barata,26.00,3.35,9.21,17.00,3.78,35.00,1.95,127.00,7.61,315.00,26.45,17.00,3.64
4_3_agrop_prog_acoes_mudas_financiado,3.00,0.39,0.00,3.00,0.67,4.00,0.22,8.00,0.48,24.00,2.02,0.00,0.00
4_3_agrop_prog_acoes_mudas_outros,25.00,3.23,5.96,9.00,2.00,52.00,2.90,108.00,6.47,138.00,11.59,24.00,5.14
4_3_agrop_prog_acoes_adubos_dist_gratuita,158.00,20.39,11.31,112.00,24.89,165.00,9.20,116.00,6.95,184.00,15.45,52.00,11.13
4_3_agrop_prog_acoes_adubos_mais_barata,16.00,2.06,3.76,9.00,2.00,11.00,0.61,68.00,4.08,107.00,8.98,14.00,3.00


#### Programa ou ação para disponibilizar maquinário aos produtores agropecuários desenvolvido pela prefeitura
CONCLUSÃO: 
- Maior parte dos programas são voltados para cessão de tempo gratuito

In [29]:
variaveis = ['4_3_agrop_prog_acoes_maquin_cessao_temp_gratu', '4_3_agrop_prog_acoes_maquin_aluguel','4_3_agrop_prog_acoes_maquin_outro','4_3_agrop_prog_acoes_maquin_nenhumo']
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_3_agrop_prog_acoes_maquin_cessao_temp_gratu,478,61.68,54.55,290,64.44,1162,64.77,698,41.85,615,51.64,270,57.82
4_3_agrop_prog_acoes_maquin_aluguel,56,7.23,17.32,26,5.78,67,3.73,510,30.58,289,24.27,71,15.20
4_3_agrop_prog_acoes_maquin_outro,186,24.00,27.07,108,24.00,305,17.00,431,25.84,513,43.07,148,31.69
4_3_agrop_prog_acoes_maquin_nenhumo,50,6.45,12.29,30,6.67,304,16.95,261,15.65,47,3.95,42,8.99


#### A prefeitura desenvolve programa ou ação de estímulo
CONCLUSÃO: 
- Há uma grande proporção de municipios participantes para hortas comunitárias (associado a subsistência). Por outro lado, aquicultura e pesca são aqueles com maior porcentual a nível nacional (média ponderada)

In [30]:
variaveis = ['4_3_agrop_prog_acoes_agric_organica','4_3_agrop_prog_acoes_agric_familiar','4_3_agrop_prog_acoes_aquicultura',
'4_3_agrop_prog_acoes_pesca','4_3_agrop_prog_acoes_hortas_comunitarias',]
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_3_agrop_prog_acoes_agric_organica,262,33.81,41.43,164,36.44,835,46.54,607,36.39,582,48.87,116,24.84
4_3_agrop_prog_acoes_agric_familiar,635,81.94,85.66,373,82.89,1577,87.90,1357,81.35,1088,91.35,370,79.23
4_3_agrop_prog_acoes_aquicultura,251,32.39,28.02,164,36.44,496,27.65,302,18.11,472,39.63,124,26.55
4_3_agrop_prog_acoes_pesca,234,30.19,20.78,153,34.00,518,28.87,177,10.61,253,21.24,55,11.78
4_3_agrop_prog_acoes_hortas_comunitarias,331,42.71,37.31,206,45.78,756,42.14,551,33.03,395,33.17,168,35.97


#### Programas para agroindústria e promoção e apoio a festividades e premiação

In [31]:
variaveis = ['4_4_agrop_promo_festiv_existencia','4_4_agrop_promo_festiv_festividades','4_4_agrop_promo_festiv_premiacoes','4_4_agrop_preven_prob_climatico']
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_4_agrop_promo_festiv_existencia,349,45.03,51.16,205,45.56,708,39.46,798,47.84,923,77.50,210,44.97
4_4_agrop_promo_festiv_festividades,458,59.10,61.18,292,64.89,957,53.34,1056,63.31,818,68.68,280,59.96
4_4_agrop_promo_festiv_premiacoes,162,20.90,22.04,109,24.22,329,18.34,431,25.84,289,24.27,68,14.56
4_4_agrop_preven_prob_climatico,298,38.45,51.59,164,36.44,1144,63.77,583,34.95,788,66.16,190,40.69


#### Apoio às atividades agropecuárias
- Órgão público que atue na assistência técnica e/ou extensão rural no município / Prestadores de serviços de assistência técnica e/ou extensão rural para o setor agropecuário, contratados ou parceiros da prefeitura

In [32]:
variaveis = ['4_5_agrop_assistec_municipal','4_5_agrop_assistec_estadual','4_5_agrop_assistec_federal','4_5_agrop_assistec_naoexiste','4_5_agrop_assistec_inst_conveniadas',
'4_5_agrop_assistec_empres_privada','4_5_agrop_assistec_ong','4_5_agrop_assistec_outro','4_5_agrop_assistec_nenhum',]
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_5_agrop_assistec_municipal,453,58.45,52.61,252,56.00,994,55.41,771,46.22,713,59.87,196,41.97
4_5_agrop_assistec_estadual,540,69.68,78.17,343,76.22,1294,72.13,1321,79.20,1054,88.50,336,71.95
4_5_agrop_assistec_federal,58,7.48,6.02,42,9.33,152,8.47,69,4.14,35,2.94,37,7.92
4_5_agrop_assistec_naoexiste,37,4.77,6.63,19,4.22,129,7.19,138,8.27,27,2.27,56,11.99
4_5_agrop_assistec_inst_conveniadas,0,0.00,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00
4_5_agrop_assistec_empres_privada,63,8.13,11.98,36,8.00,155,8.64,167,10.01,265,22.25,43,9.21
4_5_agrop_assistec_ong,90,11.61,11.26,54,12.00,238,13.27,127,7.61,164,13.77,43,9.21
4_5_agrop_assistec_outro,174,22.45,27.66,95,21.11,546,30.43,448,26.86,328,27.54,122,26.12
4_5_agrop_assistec_nenhum,408,52.65,51.88,251,55.78,889,49.55,978,58.63,501,42.07,268,57.39


#### A prefeitura desenvolve programa ou ação
- De cunho social em apoio específico ao produtor agropecuário / Para fomentar o artesanato junto às comunidades rurais

In [33]:
variaveis = [#'4_5_agrop_prog_acoes_social_existencia',
'4_5_agrop_prog_acoes_social_educa','4_5_agrop_prog_acoes_social_saude','4_5_agrop_prog_acoes_social_alimen','4_5_agrop_prog_acoes_social_outros',
#'4_5_agrop_prog_acoes_artes_existencia',
'4_5_agrop_prog_acoes_artes_capacit','4_5_agrop_prog_acoes_artes_apoio_comer','4_5_agrop_prog_acoes_artes_associatv','4_5_agrop_prog_acoes_artes_outros',]
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_5_agrop_prog_acoes_social_educa,414,53.42,51.33,238,52.89,874,48.72,844,50.60,686,57.60,213,45.61
4_5_agrop_prog_acoes_social_saude,146,18.84,17.85,96,21.33,296,16.50,289,17.33,236,19.82,76,16.27
4_5_agrop_prog_acoes_social_alimen,206,26.58,24.17,133,29.56,534,29.77,345,20.68,258,21.66,75,16.06
4_5_agrop_prog_acoes_social_outros,84,10.84,11.45,54,12.00,210,11.71,158,9.47,160,13.43,55,11.78
4_5_agrop_prog_acoes_artes_capacit,233,30.06,33.72,147,32.67,515,28.71,573,34.35,507,42.57,133,28.48
4_5_agrop_prog_acoes_artes_apoio_comer,179,23.10,24.45,119,26.44,481,26.81,393,23.56,296,24.85,71,15.20
4_5_agrop_prog_acoes_artes_associatv,191,24.65,29.79,117,26.00,532,29.65,504,30.22,405,34.01,99,21.20
4_5_agrop_prog_acoes_artes_outros,42,5.42,4.60,27,6.00,98,5.46,59,3.54,52,4.37,20,4.28


#### Apoio ao associativismo
- Entidades de associativismo ou representação de produtores agropecuários com registro oficial na prefeitura / Entidades de associativismo ou representação de produtores agropecuários que atuam no município

In [34]:
variaveis = ['4_6_agrop_associatv_cadpref_sind_rural','4_6_agrop_associatv_cadpref_assoc_prod','4_6_agrop_associatv_cadpref_cooperat','4_6_agrop_associatv_cadpref_naopossui',
'4_6_agrop_associatv_atua_munc_sind_rural','4_6_agrop_associatv_atua_munc_assoc_prod','4_6_agrop_associatv_atua_munc_cooperat','4_6_agrop_associatv_atua_munc_outro','4_6_agrop_associatv_atua_munc_nenhuma',]
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_6_agrop_associatv_cadpref_sind_rural,412,53.16,58.22,241,53.56,1044,58.19,892,53.48,787,66.08,274,58.67
4_6_agrop_associatv_cadpref_assoc_prod,476,61.42,62.38,300,66.67,1119,62.37,1029,61.69,770,64.65,252,53.96
4_6_agrop_associatv_cadpref_cooperat,243,31.35,30.49,164,36.44,386,21.52,399,23.92,592,49.71,154,32.98
4_6_agrop_associatv_cadpref_naopossui,131,16.90,20.74,70,15.56,423,23.58,380,22.78,169,14.19,112,23.98
4_6_agrop_associatv_atua_munc_sind_rural,533,68.77,76.82,304,67.56,1485,82.78,1156,69.30,987,82.87,341,73.02
4_6_agrop_associatv_atua_munc_assoc_prod,558,72.00,72.86,335,74.44,1378,76.81,1156,69.30,875,73.47,309,66.17
4_6_agrop_associatv_atua_munc_cooperat,273,35.23,39.96,183,40.67,508,28.32,546,32.73,798,67.00,186,39.83
4_6_agrop_associatv_atua_munc_outro,36,4.65,5.21,21,4.67,114,6.35,69,4.14,69,5.79,17,3.64
4_6_agrop_associatv_atua_munc_nenhuma,43,5.55,7.08,28,6.22,81,4.52,186,11.15,48,4.03,51,10.92


#### Infraestrutura
- Infraestrutura de comercialização e exposição de produtos agropecuários existente no município

In [35]:
variaveis = ['4_7_agrop_infra_centro_comercial','4_7_agrop_infra_feiras','4_7_agrop_infra_parque_exposicao','4_7_agrop_infra_c_comercial_vegetal','4_7_agrop_infra_outro','4_7_agrop_infra_nenhum',
]
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_7_agrop_infra_centro_comercial,113,14.58,15.53,69,15.33,369,20.57,184,11.03,190,15.95,52,11.13
4_7_agrop_infra_feiras,547,70.58,70.34,334,74.22,1255,69.96,1163,69.72,802,67.34,359,76.87
4_7_agrop_infra_parque_exposicao,104,13.42,13.07,62,13.78,130,7.25,267,16.01,177,14.86,91,19.49
4_7_agrop_infra_c_comercial_vegetal,32,4.13,3.58,19,4.22,76,4.24,47,2.82,42,3.53,15,3.21
4_7_agrop_infra_outro,67,8.65,8.12,44,9.78,181,10.09,120,7.19,85,7.14,22,4.71
4_7_agrop_infra_nenhum,102,13.16,19.67,50,11.11,332,18.51,372,22.30,266,22.33,74,15.85


#### Programa ou ação de aquisição de produtos e vacinação de rebanho, Serviço de Inspeção Municipal (SIM) e abatedouros

In [36]:
variaveis = ['4_8_agrop_aq_prod_direto_prod','4_8_agrop_aq_prod_atraves_entid','4_8_agrop_aq_prod_prog_aquis','4_8_agrop_aq_prod_outro','4_8_agrop_aq_prod_nenhum',
'4_8_agrop_vacina_gratuito','4_8_agrop_vacina_maisbarato','4_8_agrop_vacina_outro','4_8_agrop_vacina_nenhum',
'4_8_agrop_impl_sim','4_8_agrop_abatedouro_existencia','4_8_agrop_abatedouro_uso12meses','4_8_agrop_exist_extracao_vegetal',]
analise_brasil(dfr_munic_agropec, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
4_8_agrop_aq_prod_direto_prod,286,36.90,36.39,165,36.67,669,37.29,556,33.33,508,42.65,126,26.98
4_8_agrop_aq_prod_atraves_entid,198,25.55,26.47,125,27.78,496,27.65,385,23.08,371,31.15,95,20.34
4_8_agrop_aq_prod_prog_aquis,491,63.35,65.89,301,66.89,1195,66.61,1029,61.69,875,73.47,265,56.75
4_8_agrop_aq_prod_outro,95,12.26,10.17,63,14.00,169,9.42,188,11.27,106,8.90,40,8.57
4_8_agrop_aq_prod_nenhum,86,11.10,14.18,43,9.56,248,13.82,281,16.85,101,8.48,116,24.84
4_8_agrop_vacina_gratuito,32,4.13,13.27,12,2.67,348,19.40,190,11.39,173,14.53,15,3.21
4_8_agrop_vacina_maisbarato,29,3.74,6.53,22,4.89,94,5.24,68,4.08,169,14.19,10,2.14
4_8_agrop_vacina_outro,85,10.97,17.10,50,11.11,364,20.29,291,17.45,207,17.38,39,8.35
4_8_agrop_vacina_nenhum,553,71.35,63.21,326,72.44,980,54.63,1146,68.71,668,56.09,397,85.01
4_8_agrop_impl_sim,420,54.19,54.64,266,59.11,781,43.53,785,47.06,960,80.60,246,52.68


### 0.2.5 ABA - MEIO AMBIENTE

In [37]:
# Leitura dos dados da base fornecida
dfr_munic_ambiente = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Meio ambiente')
map_df = {'Mmam01'   : "5_1_ambiente_gest_polit_caract_orgao",
          'Mmam03'   : "5_1_ambiente_gest_polit_sexo_tit",
          'Mmam04'   : "5_1_ambiente_gest_polit_idade_tit",
          'Mmam05'   : "5_1_ambiente_gest_polit_raca_tit",
          'Mmam051'  : "5_1_ambiente_gest_polit_resp_prop_tit",
          'Mmam06'   : "5_1_ambiente_gest_polit_escolar_tit",
          'Mmam08'   : "5_2_ambiente_capac_ult4anos_govfed",
          'Mmam091'  : "5_2_ambiente_capac_area_estrutura",
          'Mmam092'  : "5_2_ambiente_capac_area_licenciam",
          'Mmam093'  : "5_2_ambiente_capac_area_edu_ambiental",
          'Mmam094'  : "5_2_ambiente_capac_area_edu_amb_agric_fam",
          'Mmam095'  : "5_2_ambiente_capac_area_cad_amb_rural",
          'Mmam096'  : "5_2_ambiente_capac_area_resid_solidos",
          'Mmam097'  : "5_2_ambiente_capac_area_prod_cons_sustent",
          'Mmam098'  : "5_2_ambiente_capac_area_mudanca_cimatica",
          'Mmam099'  : "5_2_ambiente_capac_area_recursos_hidr",
          'Mmam0910' : "5_2_ambiente_capac_area_part_foruns_coleg",
          'Mmam0911' : "5_2_ambiente_capac_area_outras",
          'Mmam10'   : "5_3_ambiente_cons_existencia",
          'Mmam101b' : "5_3_ambiente_cons_ano_cria",
          'Mmam102'  : "5_3_ambiente_cons_formacao",
          'Mmam111'  : "5_3_ambiente_cons_conselho_consultivo",
          'Mmam112'  : "5_3_ambiente_cons_conselho_deliberativo",
          'Mmam113'  : "5_3_ambiente_cons_conselho_normativo",
          'Mmam114'  : "5_3_ambiente_cons_conselho_fiscalizador",
          'Mmam12'   : "5_3_ambiente_cons_qtd_reuniao_12meses",
          'Mmam13'   : "5_3_ambiente_cons_numero_conselheiros",
          'Mmam141'  : "5_3_ambiente_cons_cap_membros_period",
          'Mmam142'  : "5_3_ambiente_cons_cap_membros_ocasin",
          'Mmam143'  : "5_3_ambiente_cons_cap_membros_nrealiza",
          'Mmam15'   : "5_3_ambiente_cons_infra_exist",
          'Mmam1511' : "5_3_ambiente_cons_infra_sala",
          'Mmam1512' : "5_3_ambiente_cons_infra_compu",
          'Mmam1513' : "5_3_ambiente_cons_infra_impres",
          'Mmam1514' : "5_3_ambiente_cons_infra_intern",
          'Mmam1515' : "5_3_ambiente_cons_infra_veicu",
          'Mmam1516' : "5_3_ambiente_cons_infra_telef",
          'Mmam1517' : "5_3_ambiente_cons_infra_diarias",
          'Mmam1518' : "5_3_ambiente_cons_infra_dot_orc_propria",
          'Mmam16'   : "5_3_ambiente_possui_rec_proprio",
          'Mmam17'   : "5_3_ambiente_fund_munic_existencia",
          'Mmam171'  : "5_3_ambiente_fund_munic_gestor_fund_cons",
          'Mmam18'   : "5_3_ambiente_fund_munic_2019_empenhado",
          'Mmam201'  : "5_4_ambiente_legis_colet_sel_existencia",
          'Mmam20011': "5_4_ambiente_legis_colet_sel_ano",
          'Mmam202'  : "5_4_ambiente_legis_san_bas_existencia",
          'Mmam2021' : "5_4_ambiente_legis_san_bas_ano",
          'Mmam203'  : "5_4_ambiente_legis_gest_bachidro_existencia",
          'Mmam2031' : "5_4_ambiente_legis_gest_bachidro_ano",
          'Mmam204'  : "5_4_ambiente_legis_z_prot_existencia",
          'Mmam2041' : "5_4_ambiente_legis_z_prot_ano",
          'Mmam205'  : "5_4_ambiente_legis_dest_emb_agrotx_existencia",
          'Mmam2051' : "5_4_ambiente_legis_dest_emb_agrotx_ano",
          'Mmam206'  : "5_4_ambiente_legis_polu_ar_existencia",
          'Mmam2061' : "5_4_ambiente_legis_polu_ar_ano",
          'Mmam207'  : "5_4_ambiente_legis_perm_ext_miner_existencia",
          'Mmam2071' : "5_4_ambiente_legis_perm_ext_miner_ano",
          'Mmam208'  : "5_4_ambiente_legis_fauna_silv_existencia",
          'Mmam2081' : "5_4_ambiente_legis_fauna_silv_ano",
          'Mmam209'  : "5_4_ambiente_legis_floresta_existencia",
          'Mmam2091' : "5_4_ambiente_legis_floresta_ano",
          'Mmam2010' : "5_4_ambiente_legis_prt_biodv_existencia",
          'Mmam20101': "5_4_ambiente_legis_prt_biodv_ano",
          'Mmam2011' : "5_4_ambiente_legis_mit_md_clima_existencia",
          'Mmam20111': "5_4_ambiente_legis_mit_md_clima_ano",
          'Mmam2012' : "5_4_ambiente_legis_nenhuma",
          'Mmam21'   : "5_5_ambiente_plano_res_solido_existencia",
          'Mmam211'  : "5_5_ambiente_plano_res_solido_ap_municip",
          'Mmam221'  : "5_6_ambiente_par_govfed_colet_educador",
          'Mmam222'  : "5_6_ambiente_par_govfed_sala_verde",
          'Mmam223'  : "5_6_ambiente_par_govfed_circ_tela_verde",
          'Mmam224'  : "5_6_ambiente_par_govfed_conf_meioambiente",
          'Mmam225'  : "5_6_ambiente_par_govfed_edu_ambiental_pgirs",
          'Mmam226'  : "5_6_ambiente_par_govfed_sust_ambiental",
          'Mmam227'  : "5_6_ambiente_par_govfed_edu_ambiental_familiar",
          'Mmam228'  : "5_6_ambiente_par_govfed_confer_nacional",
          'Mmam229'  : "5_6_ambiente_par_govfed_nenhum",
          'Mmam23'   : "5_7_ambiente_pag_serv_ambi_fonte_diretamente",
          'Mmam2311' : "5_7_ambiente_pag_serv_ambi_fonte_12m_municp",
          'Mmam2312' : "5_7_ambiente_pag_serv_ambi_fonte_12m_govfed",
          'Mmam2313' : "5_7_ambiente_pag_serv_ambi_fonte_12m_govest",
          'Mmam2314' : "5_7_ambiente_pag_serv_ambi_fonte_12m_inipriv",
          'Mmam2315' : "5_7_ambiente_pag_serv_ambi_fonte_12m_ong",
          'Mmam2316' : "5_7_ambiente_pag_serv_ambi_fonte_12m_doacoes",
          'Mmam2317' : "5_7_ambiente_pag_serv_ambi_fonte_12m_outros",
          'Mmam241'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_rec_hidrico",
          'Mmam242'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_vegt_nativa",
          'Mmam243'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_unid_conserv",
          'Mmam244'  : "5_7_ambiente_pag_serv_ambi_tipo_rec_con_cobert_vegt",
          'Mmam245'  : "5_7_ambiente_pag_serv_ambi_tipo_conserv_vegt_urbana",
          'Mmam246'  : "5_7_ambiente_pag_serv_ambi_tipo_ret_carbono",
          'Mmam247'  : "5_7_ambiente_pag_serv_ambi_tipo_outros",
          'Mmam25'   : "5_8_ambiente_impc_amb_ocorr_24m_existencia",
          'Mmam261'  : "5_8_ambiente_impc_amb_condicoes_extremas",
          'Mmam262'  : "5_8_ambiente_impc_amb_polu_ar",
          'Mmam263'  : "5_8_ambiente_impc_amb_polu_corpo_agua",
          'Mmam264'  : "5_8_ambiente_impc_amb_assoreamento",
          'Mmam265'  : "5_8_ambiente_impc_amb_diminin_vazao",
          'Mmam266'  : "5_8_ambiente_impc_amb_desmatamento",
          'Mmam267'  : "5_8_ambiente_impc_amb_queimadas",
          'Mmam268'  : "5_8_ambiente_impc_amb_contam_solo",
          'Mmam269'  : "5_8_ambiente_impc_amb_erosao_desert",
          'Mmam2610' : "5_8_ambiente_impc_amb_degr_areas_proteg",
          'Mmam2611' : "5_8_ambiente_impc_amb_diminin_biodv",
          'Mmam2612' : "5_8_ambiente_impc_amb_moradia_risco_ambiental",
          'Mmam2613' : "5_8_ambiente_impc_amb_falta_saneamento",
          'Mmam2614' : "5_8_ambiente_impc_amb_outros"
        }
dfr_munic_ambiente = dfr_munic_ambiente.rename(columns=map_df)
dfr_munic_ambiente = pd.merge(dfr_munic_ambiente, df_amz_legal, on='CodMun', how='left')

In [38]:
#dfr_munic_ambiente.to_csv('../dados/munic/2020_munic_meio_ambiente.csv', index=False)

#### Legislação ou instrumento de gestão ambiental 
- Legislação ou instrumento de gestão ambiental existente no município, mesmo que esteja inserido na Lei Orgânica, Plano Diretor, Código Ambiental, etc

In [39]:
variaveis = ['5_4_ambiente_legis_colet_sel_existencia',#'5_4_ambiente_legis_colet_sel_ano',
'5_4_ambiente_legis_san_bas_existencia',#'5_4_ambiente_legis_san_bas_ano',
'5_4_ambiente_legis_gest_bachidro_existencia',#'5_4_ambiente_legis_gest_bachidro_ano',
'5_4_ambiente_legis_z_prot_existencia',#'5_4_ambiente_legis_z_prot_ano',
'5_4_ambiente_legis_dest_emb_agrotx_existencia',#'5_4_ambiente_legis_dest_emb_agrotx_ano',
'5_4_ambiente_legis_polu_ar_existencia',#'5_4_ambiente_legis_polu_ar_ano',
'5_4_ambiente_legis_perm_ext_miner_existencia',#'5_4_ambiente_legis_perm_ext_miner_ano',
'5_4_ambiente_legis_fauna_silv_existencia',#'5_4_ambiente_legis_fauna_silv_ano',
'5_4_ambiente_legis_floresta_existencia',#'5_4_ambiente_legis_floresta_ano',
'5_4_ambiente_legis_prt_biodv_existencia',#'5_4_ambiente_legis_prt_biodv_ano',
'5_4_ambiente_legis_mit_md_clima_existencia',#'5_4_ambiente_legis_mit_md_clima_ano',
'5_4_ambiente_legis_nenhuma',]
analise_brasil(dfr_munic_ambiente, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
5_4_ambiente_legis_colet_sel_existencia,345,44.52,56.70,233,51.78,746,41.58,1001,60.01,891,74.81,282,60.39
5_4_ambiente_legis_san_bas_existencia,431,55.61,66.26,262,58.22,842,46.93,1262,75.66,1003,84.21,316,67.67
5_4_ambiente_legis_gest_bachidro_existencia,152,19.61,20.14,93,20.67,275,15.33,371,22.24,265,22.25,116,24.84
5_4_ambiente_legis_z_prot_existencia,297,38.32,41.08,192,42.67,588,32.78,722,43.29,537,45.09,246,52.68
5_4_ambiente_legis_dest_emb_agrotx_existencia,167,21.55,21.28,110,24.44,281,15.66,326,19.54,347,29.14,119,25.48
5_4_ambiente_legis_polu_ar_existencia,263,33.94,33.49,165,36.67,474,26.42,655,39.27,370,31.07,199,42.61
5_4_ambiente_legis_perm_ext_miner_existencia,215,27.74,24.68,146,32.44,441,24.58,361,21.64,283,23.76,142,30.41
5_4_ambiente_legis_fauna_silv_existencia,247,31.87,26.50,160,35.56,460,25.64,376,22.54,315,26.45,163,34.90
5_4_ambiente_legis_floresta_existencia,243,31.35,28.46,161,35.78,439,24.47,418,25.06,397,33.33,168,35.97
5_4_ambiente_legis_prt_biodv_existencia,241,31.10,27.86,161,35.78,479,26.70,414,24.82,311,26.11,185,39.61


#### Plano de Gestão Integrada de Resíduos Sólidos
- O município possui Plano de Gestão Integrada de Resíduos Sólidos, nos termos da Política Nacional de Resíduos Sólidos

In [40]:
variaveis = ['5_5_ambiente_plano_res_solido_existencia',
'5_5_ambiente_plano_res_solido_ap_municip',]
analise_brasil(dfr_munic_ambiente, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
5_5_ambiente_plano_res_solido_existencia,414,53.42,66.71,271,60.22,948,52.84,1173,70.32,1008,84.63,310,66.38
5_5_ambiente_plano_res_solido_ap_municip,364,46.97,53.62,247,54.89,610,34.00,1073,64.33,824,69.19,228,48.82


#### Programas em parceria com o Governo Federal
- Programas implementados pelo governo municipal em parceria com o Governo Federal

In [41]:
variaveis = ['5_6_ambiente_par_govfed_colet_educador','5_6_ambiente_par_govfed_sala_verde','5_6_ambiente_par_govfed_circ_tela_verde',
'5_6_ambiente_par_govfed_conf_meioambiente','5_6_ambiente_par_govfed_edu_ambiental_pgirs','5_6_ambiente_par_govfed_sust_ambiental',
'5_6_ambiente_par_govfed_edu_ambiental_familiar','5_6_ambiente_par_govfed_confer_nacional','5_6_ambiente_par_govfed_nenhum',]
analise_brasil(dfr_munic_ambiente, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
5_6_ambiente_par_govfed_colet_educador,6,0.77,2.05,4,0.89,24,1.34,24,1.44,42,3.53,20,4.28
5_6_ambiente_par_govfed_sala_verde,31,4.00,5.91,19,4.22,96,5.35,131,7.85,57,4.79,26,5.57
5_6_ambiente_par_govfed_circ_tela_verde,10,1.29,1.72,10,2.22,36,2.01,27,1.62,14,1.18,9,1.93
5_6_ambiente_par_govfed_conf_meioambiente,17,2.19,3.47,10,2.22,98,5.46,35,2.10,37,3.11,13,2.78
5_6_ambiente_par_govfed_edu_ambiental_pgirs,93,12.00,11.97,65,14.44,195,10.87,171,10.25,167,14.02,68,14.56
5_6_ambiente_par_govfed_sust_ambiental,25,3.23,4.49,14,3.11,95,5.30,83,4.98,25,2.10,33,7.07
5_6_ambiente_par_govfed_edu_ambiental_familiar,86,11.10,8.37,58,12.89,173,9.64,103,6.18,80,6.72,52,11.13
5_6_ambiente_par_govfed_confer_nacional,49,6.32,4.96,38,8.44,115,6.41,45,2.70,47,3.95,31,6.64
5_6_ambiente_par_govfed_nenhum,484,62.45,71.60,275,61.11,1233,68.73,1273,76.32,904,75.90,298,63.81


#### Pagamento de Serviços Ambientais
- O município paga diretamente por serviços ambientais – PSA (Fonte de recursos nos últimos 12 meses)
- Serviço(s) ambiental(is) abrangido(s)

In [42]:
variaveis = ['5_7_ambiente_pag_serv_ambi_fonte_diretamente','5_7_ambiente_pag_serv_ambi_fonte_12m_municp','5_7_ambiente_pag_serv_ambi_fonte_12m_govfed',
'5_7_ambiente_pag_serv_ambi_fonte_12m_govest','5_7_ambiente_pag_serv_ambi_fonte_12m_inipriv','5_7_ambiente_pag_serv_ambi_fonte_12m_ong',
'5_7_ambiente_pag_serv_ambi_fonte_12m_doacoes','5_7_ambiente_pag_serv_ambi_fonte_12m_outros',
'5_7_ambiente_pag_serv_ambi_tipo_rec_con_rec_hidrico',
'5_7_ambiente_pag_serv_ambi_tipo_rec_con_vegt_nativa','5_7_ambiente_pag_serv_ambi_tipo_rec_con_unid_conserv','5_7_ambiente_pag_serv_ambi_tipo_rec_con_cobert_vegt',
'5_7_ambiente_pag_serv_ambi_tipo_conserv_vegt_urbana','5_7_ambiente_pag_serv_ambi_tipo_ret_carbono','5_7_ambiente_pag_serv_ambi_tipo_outros',]
analise_brasil(dfr_munic_ambiente, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
5_7_ambiente_pag_serv_ambi_fonte_diretamente,87.00,11.23,15.04,54,12.00,220.00,12.26,280.00,16.79,161.00,13.52,122.00,26.12
5_7_ambiente_pag_serv_ambi_fonte_12m_municp,76.00,9.81,12.51,46,10.22,192.00,10.70,221.00,13.25,140.00,11.75,97.00,20.77
5_7_ambiente_pag_serv_ambi_fonte_12m_govfed,7.00,0.90,1.13,5,1.11,20.00,1.11,17.00,1.02,11.00,0.92,10.00,2.14
5_7_ambiente_pag_serv_ambi_fonte_12m_govest,20.00,2.58,1.96,19,4.22,22.00,1.23,28.00,1.68,17.00,1.43,23.00,4.93
5_7_ambiente_pag_serv_ambi_fonte_12m_inipriv,2.00,0.26,0.74,1,0.22,13.00,0.72,11.00,0.66,10.00,0.84,6.00,1.28
5_7_ambiente_pag_serv_ambi_fonte_12m_ong,1.00,0.13,0.00,0,0.00,3.00,0.17,6.00,0.36,4.00,0.34,3.00,0.64
5_7_ambiente_pag_serv_ambi_fonte_12m_doacoes,4.00,0.52,0.74,2,0.44,7.00,0.39,16.00,0.96,5.00,0.42,11.00,2.36
5_7_ambiente_pag_serv_ambi_fonte_12m_outros,5.00,0.65,1.71,1,0.22,23.00,1.28,37.00,2.22,21.00,1.76,13.00,2.78
5_7_ambiente_pag_serv_ambi_tipo_rec_con_rec_hidrico,29.00,3.74,5.82,18,4.00,75.00,4.18,125.00,7.49,54.00,4.53,52.00,11.13
5_7_ambiente_pag_serv_ambi_tipo_rec_con_vegt_nativa,23.00,2.97,3.41,17,3.78,37.00,2.06,68.00,4.08,35.00,2.94,33.00,7.07


#### Impacto Ambiental e/ou processo/ação que resulte em impacto no ambiente

In [43]:
variaveis = ['5_8_ambiente_impc_amb_ocorr_24m_existencia','5_8_ambiente_impc_amb_condicoes_extremas','5_8_ambiente_impc_amb_polu_ar','5_8_ambiente_impc_amb_polu_corpo_agua',
'5_8_ambiente_impc_amb_assoreamento','5_8_ambiente_impc_amb_diminin_vazao','5_8_ambiente_impc_amb_desmatamento','5_8_ambiente_impc_amb_queimadas',
'5_8_ambiente_impc_amb_contam_solo','5_8_ambiente_impc_amb_erosao_desert','5_8_ambiente_impc_amb_degr_areas_proteg','5_8_ambiente_impc_amb_diminin_biodv',
'5_8_ambiente_impc_amb_moradia_risco_ambiental','5_8_ambiente_impc_amb_falta_saneamento','5_8_ambiente_impc_amb_outros',]
analise_brasil(dfr_munic_ambiente, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
5_8_ambiente_impc_amb_ocorr_24m_existencia,532,68.65,65.04,321,71.33,1139,63.49,1053,63.13,786,65.99,319,68.31
5_8_ambiente_impc_amb_condicoes_extremas,124,16.00,26.58,63,14.00,469,26.14,387,23.20,459,38.54,100,21.41
5_8_ambiente_impc_amb_polu_ar,85,10.97,6.34,53,11.78,126,7.02,92,5.52,46,3.86,36,7.71
5_8_ambiente_impc_amb_polu_corpo_agua,111,14.32,14.72,65,14.44,278,15.50,247,14.81,168,14.11,61,13.06
5_8_ambiente_impc_amb_assoreamento,95,12.26,11.70,50,11.11,160,8.92,278,16.67,94,7.89,69,14.78
5_8_ambiente_impc_amb_diminin_vazao,91,11.74,16.09,46,10.22,231,12.88,310,18.59,220,18.47,88,18.84
5_8_ambiente_impc_amb_desmatamento,230,29.68,19.67,152,33.78,419,23.36,167,10.01,260,21.83,96,20.56
5_8_ambiente_impc_amb_queimadas,396,51.10,32.13,235,52.22,569,31.72,608,36.45,145,12.17,232,49.68
5_8_ambiente_impc_amb_contam_solo,21,2.71,2.99,15,3.33,60,3.34,22,1.32,55,4.62,14,3.00
5_8_ambiente_impc_amb_erosao_desert,20,2.58,4.93,11,2.44,59,3.29,110,6.59,61,5.12,33,7.07


### 0.2.6 ABA - GESTÃO DE RISCO E DESASTRES

In [44]:
# Leitura dos dados da base fornecida
dfr_munic_gst_risco = pd.read_excel('../data/MUNIC/Base_MUNIC_2020.xlsx', sheet_name='Gestão de riscos')
map_df = {'Mgrd01'   : "6_1_risco_destr_seca_exist_4anos",
          'Mgrd02'   : "6_1_risco_destr_seca_maior_impc_ano",
          'Mgrd031'  : "6_1_risco_destr_seca_maior_impc_perdas_finan",
          'Mgrd032'  : "6_1_risco_destr_seca_maior_impc_perdas_human",
          'Mgrd033'  : "6_1_risco_destr_seca_maior_impc_perdas_animais",
          'Mgrd034'  : "6_1_risco_destr_seca_maior_impc_perdas_ambient",
          'Mgrd035'  : "6_1_risco_destr_seca_maior_impc_perd_red_agricult",
          'Mgrd036'  : "6_1_risco_destr_seca_maior_impc_desertificacao",
          'Mgrd037'  : "6_1_risco_destr_seca_maior_impc_poluentes_agua",
          'Mgrd038'  : "6_1_risco_destr_seca_maior_impc_outras",
          'Mgrd039'  : "6_1_risco_destr_seca_maior_impc_nao_sabe",
          'Mgrd041'  : "6_1_risco_destr_seca_acoes_corr_cisternas",
          'Mgrd042'  : "6_1_risco_destr_seca_acoes_corr_acudes",
          'Mgrd043'  : "6_1_risco_destr_seca_acoes_corr_barragens",
          'Mgrd044'  : "6_1_risco_destr_seca_acoes_corr_pocos",
          'Mgrd045'  : "6_1_risco_destr_seca_acoes_corr_revegetacao",
          'Mgrd046'  : "6_1_risco_destr_seca_acoes_corr_agrc_adap_clima",
          'Mgrd047'  : "6_1_risco_destr_seca_acoes_corr_caminhao_pipa",
          'Mgrd048'  : "6_1_risco_destr_seca_acoes_corr_uso_sust_rec_natural",
          'Mgrd049'  : "6_1_risco_destr_seca_acoes_corr_outras",
          'Mgrd0410' : "6_1_risco_destr_seca_acoes_corr_naosabe",
          'Mgrd05'   : "6_1_risco_destr_seca_plano_contigencia",
          'Mgrd06'   : "6_2_risco_destr_alagamento_4anos_exist",
          'Mgrd07'   : "6_2_risco_destr_erosao_acel_4anos_exist",
          'Mgrd08'   : "6_3_risco_destr_enchente_4anos_exist",
          'Mgrd09'   : "6_3_risco_destr_enchente_maior_impac_ano",
          'Mgrd101'  : "6_3_risco_destr_enchente_maior_impac_edif",
          'Mgrd1011' : "6_3_risco_destr_enchente_maior_impac_edif_area",
          'Mgrd102'  : "6_3_risco_destr_enchente_maior_impac_pess",
          'Mgrd1021' : "6_3_risco_destr_enchente_maior_impac_pess_area_deslj",
          'Mgrd103'  : "6_3_risco_destr_enchente_maior_impac_obitos",
          'Mgrd1031' : "6_3_risco_destr_enchente_maior_impac_obitos_area",
          'Mgrd1041' : "6_3_risco_destr_enchente_area_nat_inund",
          'Mgrd1042' : "6_3_risco_destr_enchente_area_nao_inund",
          'Mgrd1043' : "6_3_risco_destr_enchente_area_ocp_regular",
          'Mgrd1044' : "6_3_risco_destr_enchente_area_ocp_irregul",
          'Mgrd1045' : "6_3_risco_destr_enchente_area_proc_erosiv",
          'Mgrd1046' : "6_3_risco_destr_enchente_area_outros",
          'Mgrd1047' : "6_3_risco_destr_enchente_area_naosabe",
          'Mgrd1051' : "6_3_risco_destr_enchente_acoes_corrt_barragem",
          'Mgrd1052' : "6_3_risco_destr_enchente_acoes_corrt_canais",
          'Mgrd1053' : "6_3_risco_destr_enchente_acoes_corrt_parque",
          'Mgrd1054' : "6_3_risco_destr_enchente_acoes_corrt_reserv",
          'Mgrd1055' : "6_3_risco_destr_enchente_acoes_corrt_desassor",
          'Mgrd1056' : "6_3_risco_destr_enchente_acoes_corrt_retific_rios",
          'Mgrd1057' : "6_3_risco_destr_enchente_acoes_corrt_realoc_pop",
          'Mgrd1058' : "6_3_risco_destr_enchente_acoes_corrt_revegetacao",
          'Mgrd1059' : "6_3_risco_destr_enchente_acoes_corrt_rev_rios_bacias",
          'Mgrd10510': "6_3_risco_destr_enchente_acoes_corrt_outras_solu",
          'Mgrd10511': "6_3_risco_destr_enchente_acoes_corrt_nenhuma",
          'Mgrd10512': "6_3_risco_destr_enchente_acoes_corrt_nao_sabe",
          'Mgrd11'   : "6_4_risco_destr_enxurrada_4anos_exist",
          'Mgrd12'   : "6_4_risco_destr_enxurrada_maior_impac_ano",
          'Mgrd131'  : "6_4_risco_destr_enxurrada_maior_impac_edif",
          'Mgrd1311' : "6_4_risco_destr_enxurrada_maior_impac_edif_area",
          'Mgrd132'  : "6_4_risco_destr_enxurrada_maior_impac_pess",
          'Mgrd1321' : "6_4_risco_destr_enxurrada_maior_impac_pess_area_deslj",
          'Mgrd133'  : "6_4_risco_destr_enxurrada_maior_impac_obitos",
          'Mgrd1331' : "6_4_risco_destr_enxurrada_maior_impac_obitos_area",
          'Mgrd1341' : "6_4_risco_destr_enxurrada_area_nat_inund",
          'Mgrd1342' : "6_4_risco_destr_enxurrada_area_nao_inund",
          'Mgrd1343' : "6_4_risco_destr_enxurrada_area_ocp_regular",
          'Mgrd1344' : "6_4_risco_destr_enxurrada_area_ocp_irregul",
          'Mgrd1345' : "6_4_risco_destr_enxurrada_area_proc_erosiv",
          'Mgrd1346' : "6_4_risco_destr_enxurrada_area_outros",
          'Mgrd1347' : "6_4_risco_destr_enxurrada_area_naosabe",
          'Mgrd1351' : "6_4_risco_destr_enxurrada_acoes_corrt_barragem",
          'Mgrd1352' : "6_4_risco_destr_enxurrada_acoes_corrt_canais",
          'Mgrd1353' : "6_4_risco_destr_enxurrada_acoes_corrt_parque",
          'Mgrd1354' : "6_4_risco_destr_enxurrada_acoes_corrt_reserv",
          'Mgrd1355' : "6_4_risco_destr_enxurrada_acoes_corrt_desassor",
          'Mgrd1356' : "6_4_risco_destr_enxurrada_acoes_corrt_retific_rios",
          'Mgrd1357' : "6_4_risco_destr_enxurrada_acoes_corrt_realoc_pop",
          'Mgrd1358' : "6_4_risco_destr_enxurrada_acoes_corrt_revegetacao",
          'Mgrd1359' : "6_4_risco_destr_enxurrada_acoes_corrt_rev_rios_bacias",
          'Mgrd13510': "6_4_risco_destr_enxurrada_acoes_corrt_outras_solu",
          'Mgrd13511': "6_4_risco_destr_enxurrada_acoes_corrt_nenhuma",
          'Mgrd13512': "6_4_risco_destr_enxurrada_acoes_corrt_nao_sabe",
          'Mgrd14'   : "6_5_risco_destr_deslizam_4anos_exist",
          'Mgrd15'   : "6_5_risco_destr_deslizam_maior_impac_ano",
          'Mgrd161'  : "6_5_risco_destr_deslizam_maior_impac_edif",
          'Mgrd1611' : "6_5_risco_destr_deslizam_maior_impac_edif_area",
          'Mgrd162'  : "6_5_risco_destr_deslizam_maior_impac_pess",
          'Mgrd1621' : "6_5_risco_destr_deslizam_maior_impac_pess_area_deslj",
          'Mgrd163'  : "6_5_risco_destr_deslizam_maior_impac_obitos",
          'Mgrd1631' : "6_5_risco_destr_deslizam_maior_impac_obitos_area",
          'Mgrd1641' : "6_5_risco_destr_deslizam_area_talude",
          'Mgrd1642' : "6_5_risco_destr_deslizam_area_erosao_cronico",
          'Mgrd1643' : "6_5_risco_destr_deslizam_area_sem_drenagem",
          'Mgrd1644' : "6_5_risco_destr_deslizam_area_ocp_regular",
          'Mgrd1645' : "6_5_risco_destr_deslizam_area_ocp_irregul",
          'Mgrd1646' : "6_5_risco_destr_deslizam_area_sem_ocp",
          'Mgrd1647' : "6_5_risco_destr_deslizam_area_outras_areas",
          'Mgrd1648' : "6_5_risco_destr_deslizam_area_naosabe",
          'Mgrd1651' : "6_5_risco_destr_deslizam_acoes_corrt_est_talude",
          'Mgrd1652' : "6_5_risco_destr_deslizam_acoes_corrt_retaludamento",
          'Mgrd1653' : "6_5_risco_destr_deslizam_acoes_corrt_contencao",
          'Mgrd1654' : "6_5_risco_destr_deslizam_acoes_corrt_macrodrenagens",
          'Mgrd1655' : "6_5_risco_destr_deslizam_acoes_corrt_realoc_pop",
          'Mgrd1656' : "6_5_risco_destr_deslizam_acoes_corrt_reveg_encosta",
          'Mgrd1657' : "6_5_risco_destr_deslizam_acoes_corrt_outra_solucao",
          'Mgrd1658' : "6_5_risco_destr_deslizam_acoes_corrt_nenhuma_interv",
          'Mgrd1659' : "6_5_risco_destr_deslizam_acoes_corrt_nao_sabe",
          'Mgrd171'  : "6_6_risco_destr_inst_planej_pde_enchente",
          'Mgrd172'  : "6_6_risco_destr_inst_planej_lpuos_enchente",
          'Mgrd173'  : "6_6_risco_destr_inst_planej_lei_esp_enchente",
          'Mgrd174'  : "6_6_risco_destr_inst_planej_pde_deslizamento",
          'Mgrd175'  : "6_6_risco_destr_inst_planej_lpuos_deslizamento",
          'Mgrd176'  : "6_6_risco_destr_inst_planej_lei_esp_deslizamento",
          'Mgrd177'  : "6_6_risco_destr_inst_planej_reducao_ruido",
          'Mgrd178'  : "6_6_risco_destr_inst_planej_carta_geotecnica",
          'Mgrd179'  : "6_6_risco_destr_inst_planej_plano_impln_obra",
          'Mgrd1710' : "6_6_risco_destr_inst_planej_nenhum",
          'Mgrd181'  : "6_6_risco_destr_grisco_enchente_map_area_enchente",
          'Mgrd182'  : "6_6_risco_destr_grisco_enchente_prog_hab_realocamento",
          'Mgrd183'  : "6_6_risco_destr_grisco_enchente_fiscaliz_area_risco",
          'Mgrd184'  : "6_6_risco_destr_grisco_enchente_plano_contigencia",
          'Mgrd185'  : "6_6_risco_destr_grisco_enchente_projeto_eng",
          'Mgrd186'  : "6_6_risco_destr_grisco_enchente_sistema_alerta",
          'Mgrd187'  : "6_6_risco_destr_grisco_enchente_cadastro_risco",
          'Mgrd188'  : "6_6_risco_destr_grisco_enchente_nenhum",
          'Mgrd19'   : "6_6_risco_destr_grisco_enchente_limpeza_bueiro",
          'Mgrd201'  : "6_6_risco_destr_grisco_delizam_map_area_enchente",
          'Mgrd202'  : "6_6_risco_destr_grisco_delizam_prog_hab_realocamento",
          'Mgrd203'  : "6_6_risco_destr_grisco_delizam_fiscaliz_area_risco",
          'Mgrd204'  : "6_6_risco_destr_grisco_delizam_plano_contigencia",
          'Mgrd205'  : "6_6_risco_destr_grisco_delizam_projeto_eng",
          'Mgrd206'  : "6_6_risco_destr_grisco_delizam_sistema_alerta",
          'Mgrd207'  : "6_6_risco_destr_grisco_delizam_cadastro_risco",
          'Mgrd208'  : "6_6_risco_destr_grisco_delizam_nenhum",
          'Mgrd211'  : "6_6_risco_destr_grisco_bombeiro",
          'Mgrd212'  : "6_6_risco_destr_grisco_coord_defesa_civil",
          'Mgrd213'  : "6_6_risco_destr_grisco_nucleo_defesa_civil",
          'Mgrd214'  : "6_6_risco_destr_grisco_guarda_municp",
          'Mgrd215'  : "6_6_risco_destr_grisco_nenhuma",
          'Mgrd216'  : "6_6_risco_destr_grisco_naosabe",
          'Mgrd221'  : "6_6_risco_destr_grdcivil_respns_exclusivo",
          'Mgrd222'  : "6_6_risco_destr_grdcivil_articulado",
          'Mgrd223'  : "6_6_risco_destr_grdcivil_usa_sist_desastre",
          'Mgrd224'  : "6_6_risco_destr_grdcivil_sist_proprio_desastre",
          'Mgrd225'  : "6_6_risco_destr_grdcivil_prev_orcamentaria",
          'Mgrd226'  : "6_6_risco_destr_grdcivil_outras_fontes_finan",
          'Mgrd227'  : "6_6_risco_destr_grdcivil_esp_fisc_exclusivo",
          'Mgrd228'  : "6_6_risco_destr_grdcivil_exist_viatura",
          'Mgrd229'  : "6_6_risco_destr_grdcivil_exist_pc",
          'Mgrd2210' : "6_6_risco_destr_grdcivil_exist_gps",
          'Mgrd2211' : "6_6_risco_destr_grdcivil_exist_coletes",
          'Mgrd2212' : "6_6_risco_destr_grdcivil_exist_serv_vinc_empreg",
          'Mgrd2213' : "6_6_risco_destr_grdcivil_exist_sistm_alerta_antecip",
          'Mgrd2214' : "6_6_risco_destr_grdcivil_nenhum",
          'Mgrd2215' : "6_6_risco_destr_grdcivil_naosabe",
          'Mgrd231'  : "6_6_risco_destr_protcivil_prat_educ_conscit_risco_escola",
          'Mgrd232'  : "6_6_risco_destr_protcivil_prat_educ_conscit_risco_comunid",
          'Mgrd233'  : "6_6_risco_destr_protcivil_educ_ambient_escolas",
          'Mgrd234'  : "6_6_risco_destr_protcivil_educ_ambient_comunid",
          'Mgrd235'  : "6_6_risco_destr_protcivil_campa_consct_risco_desatres",
          'Mgrd236'  : "6_6_risco_destr_protcivil_capct_prof_prtcivil_defcivil",
          'Mgrd237'  : "6_6_risco_destr_protcivil_form_nucleo_form_civil",
          'Mgrd238'  : "6_6_risco_destr_protcivil_nenhum",
          'Mgrd239'  : "6_6_risco_destr_protcivil_naosabe"
        }
dfr_munic_gst_risco = dfr_munic_gst_risco.rename(columns=map_df)
dfr_munic_gst_risco = pd.merge(dfr_munic_gst_risco, df_amz_legal, on='CodMun', how='left')

In [45]:
#dfr_munic_gst_risco.to_csv('../dados/munic/2020_munic_gst_risco_desastre.csv', index=False)

In [46]:
dfr_munic_gst_risco.columns.tolist()

['CodMun',
 'UF',
 'Cod UF',
 'Mun',
 'Faixa_pop',
 'Regiao',
 '6_1_risco_destr_seca_exist_4anos',
 '6_1_risco_destr_seca_maior_impc_ano',
 '6_1_risco_destr_seca_maior_impc_perdas_finan',
 '6_1_risco_destr_seca_maior_impc_perdas_human',
 '6_1_risco_destr_seca_maior_impc_perdas_animais',
 '6_1_risco_destr_seca_maior_impc_perdas_ambient',
 '6_1_risco_destr_seca_maior_impc_perd_red_agricult',
 '6_1_risco_destr_seca_maior_impc_desertificacao',
 '6_1_risco_destr_seca_maior_impc_poluentes_agua',
 '6_1_risco_destr_seca_maior_impc_outras',
 '6_1_risco_destr_seca_maior_impc_nao_sabe',
 '6_1_risco_destr_seca_acoes_corr_cisternas',
 '6_1_risco_destr_seca_acoes_corr_acudes',
 '6_1_risco_destr_seca_acoes_corr_barragens',
 '6_1_risco_destr_seca_acoes_corr_pocos',
 '6_1_risco_destr_seca_acoes_corr_revegetacao',
 '6_1_risco_destr_seca_acoes_corr_agrc_adap_clima',
 '6_1_risco_destr_seca_acoes_corr_caminhao_pipa',
 '6_1_risco_destr_seca_acoes_corr_uso_sust_rec_natural',
 '6_1_risco_destr_seca_acoes_corr

#### Seca
- O município foi atingido pela seca nos últimos 4 anos
- O que está sendo feito no município para evitar ou minimizar os danos causados pela seca

In [49]:
variaveis = ['6_1_risco_destr_seca_exist_4anos',#'6_1_risco_destr_seca_maior_impc_ano',
'6_1_risco_destr_seca_maior_impc_perdas_finan','6_1_risco_destr_seca_maior_impc_perdas_human',
'6_1_risco_destr_seca_maior_impc_perdas_animais','6_1_risco_destr_seca_maior_impc_perdas_ambient','6_1_risco_destr_seca_maior_impc_perd_red_agricult',
'6_1_risco_destr_seca_maior_impc_desertificacao','6_1_risco_destr_seca_maior_impc_poluentes_agua','6_1_risco_destr_seca_maior_impc_outras',
'6_1_risco_destr_seca_maior_impc_nao_sabe','6_1_risco_destr_seca_acoes_corr_cisternas','6_1_risco_destr_seca_acoes_corr_acudes',
'6_1_risco_destr_seca_acoes_corr_barragens','6_1_risco_destr_seca_acoes_corr_pocos','6_1_risco_destr_seca_acoes_corr_revegetacao',
'6_1_risco_destr_seca_acoes_corr_agrc_adap_clima','6_1_risco_destr_seca_acoes_corr_caminhao_pipa','6_1_risco_destr_seca_acoes_corr_uso_sust_rec_natural',
'6_1_risco_destr_seca_acoes_corr_outras','6_1_risco_destr_seca_acoes_corr_naosabe',
'6_1_risco_destr_seca_plano_contigencia',]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_1_risco_destr_seca_exist_4anos,218,28.13,52.44,124,27.56,1145,63.82,619,37.11,849,71.28,179,38.33
6_1_risco_destr_seca_maior_impc_perdas_finan,127,16.39,37.67,69,15.33,873,48.66,361,21.64,687,57.68,104,22.27
6_1_risco_destr_seca_maior_impc_perdas_human,6,0.77,1.19,4,0.89,23,1.28,17,1.02,17,1.43,5,1.07
6_1_risco_destr_seca_maior_impc_perdas_animais,90,11.61,25.91,44,9.78,821,45.76,280,16.79,210,17.63,87,18.63
6_1_risco_destr_seca_maior_impc_perdas_ambient,105,13.55,26.64,48,10.67,609,33.95,373,22.36,348,29.22,104,22.27
6_1_risco_destr_seca_maior_impc_perd_red_agricult,139,17.94,42.72,83,18.44,1002,55.85,434,26.02,754,63.31,102,21.84
6_1_risco_destr_seca_maior_impc_desertificacao,15,1.94,8.17,6,1.33,314,17.50,95,5.70,26,2.18,14,3.00
6_1_risco_destr_seca_maior_impc_poluentes_agua,24,3.10,6.90,15,3.33,169,9.42,101,6.06,83,6.97,16,3.43
6_1_risco_destr_seca_maior_impc_outras,23,2.97,4.31,11,2.44,87,4.85,65,3.90,55,4.62,22,4.71
6_1_risco_destr_seca_maior_impc_nao_sabe,6,0.77,0.63,1,0.22,6,0.33,13,0.78,9,0.76,6,1.28


#### Alagamentos e processo erosivo

In [50]:
variaveis = ['6_2_risco_destr_alagamento_4anos_exist', '6_2_risco_destr_erosao_acel_4anos_exist',]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_2_risco_destr_alagamento_4anos_exist,290,37.42,35.20,164,36.44,512,28.54,720,43.17,452,37.95,110,23.55
6_2_risco_destr_erosao_acel_4anos_exist,201,25.94,26.33,111,24.67,398,22.19,588,35.25,225,18.89,143,30.62


#### Enchente ou inundação gradual

In [51]:
variaveis = ['6_3_risco_destr_enchente_4anos_exist',#'6_3_risco_destr_enchente_maior_impac_ano',
'6_3_risco_destr_enchente_maior_impac_edif',#'6_3_risco_destr_enchente_maior_impac_edif_area',
'6_3_risco_destr_enchente_maior_impac_pess', #'6_3_risco_destr_enchente_maior_impac_pess_area_deslj',
'6_3_risco_destr_enchente_maior_impac_obitos',#'6_3_risco_destr_enchente_maior_impac_obitos_area',
'6_3_risco_destr_enchente_area_nat_inund','6_3_risco_destr_enchente_area_nao_inund', '6_3_risco_destr_enchente_area_ocp_regular',
'6_3_risco_destr_enchente_area_ocp_irregul','6_3_risco_destr_enchente_area_proc_erosiv','6_3_risco_destr_enchente_area_outros',
'6_3_risco_destr_enchente_area_naosabe',
'6_3_risco_destr_enchente_acoes_corrt_barragem','6_3_risco_destr_enchente_acoes_corrt_canais',
'6_3_risco_destr_enchente_acoes_corrt_parque','6_3_risco_destr_enchente_acoes_corrt_reserv','6_3_risco_destr_enchente_acoes_corrt_desassor',
'6_3_risco_destr_enchente_acoes_corrt_retific_rios','6_3_risco_destr_enchente_acoes_corrt_realoc_pop','6_3_risco_destr_enchente_acoes_corrt_revegetacao',
'6_3_risco_destr_enchente_acoes_corrt_rev_rios_bacias','6_3_risco_destr_enchente_acoes_corrt_outras_solu','6_3_risco_destr_enchente_acoes_corrt_nenhuma',
'6_3_risco_destr_enchente_acoes_corrt_nao_sabe',]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_3_risco_destr_enchente_4anos_exist,299,38.58,32.21,181,40.22,443,24.69,672,40.29,388,32.58,108,23.13
6_3_risco_destr_enchente_maior_impac_edif,258,33.29,27.32,157,34.89,379,21.13,592,35.49,306,25.69,86,18.42
6_3_risco_destr_enchente_maior_impac_pess,196,25.29,20.67,127,28.22,314,17.50,460,27.58,200,16.79,49,10.49
6_3_risco_destr_enchente_maior_impac_obitos,21,2.71,1.49,17,3.78,19,1.06,41,2.46,5,0.42,1,0.21
6_3_risco_destr_enchente_area_nat_inund,214,27.61,23.17,129,28.67,297,16.56,513,30.76,274,23.01,76,16.27
6_3_risco_destr_enchente_area_nao_inund,68,8.77,7.78,40,8.89,111,6.19,170,10.19,86,7.22,26,5.57
6_3_risco_destr_enchente_area_ocp_regular,76,9.81,9.31,42,9.33,115,6.41,217,13.01,123,10.33,21,4.50
6_3_risco_destr_enchente_area_ocp_irregul,105,13.55,11.29,69,15.33,171,9.53,234,14.03,117,9.82,37,7.92
6_3_risco_destr_enchente_area_proc_erosiv,21,2.71,2.17,11,2.44,37,2.06,51,3.06,12,1.01,10,2.14
6_3_risco_destr_enchente_area_outros,23,2.97,2.55,15,3.33,46,2.56,51,3.06,26,2.18,4,0.86


#### Enxurrada ou inundação brusca

In [52]:
variaveis = ['6_4_risco_destr_enxurrada_4anos_exist',#'6_4_risco_destr_enxurrada_maior_impac_ano',
'6_4_risco_destr_enxurrada_maior_impac_edif',#'6_4_risco_destr_enxurrada_maior_impac_edif_area',
'6_4_risco_destr_enxurrada_maior_impac_pess',#'6_4_risco_destr_enxurrada_maior_impac_pess_area_deslj',
'6_4_risco_destr_enxurrada_maior_impac_obitos',#'6_4_risco_destr_enxurrada_maior_impac_obitos_area',
'6_4_risco_destr_enxurrada_area_nat_inund','6_4_risco_destr_enxurrada_area_nao_inund','6_4_risco_destr_enxurrada_area_ocp_regular',
'6_4_risco_destr_enxurrada_area_ocp_irregul','6_4_risco_destr_enxurrada_area_proc_erosiv','6_4_risco_destr_enxurrada_area_outros',
'6_4_risco_destr_enxurrada_area_naosabe',
'6_4_risco_destr_enxurrada_acoes_corrt_barragem','6_4_risco_destr_enxurrada_acoes_corrt_canais',
'6_4_risco_destr_enxurrada_acoes_corrt_parque','6_4_risco_destr_enxurrada_acoes_corrt_reserv','6_4_risco_destr_enxurrada_acoes_corrt_desassor',
'6_4_risco_destr_enxurrada_acoes_corrt_retific_rios','6_4_risco_destr_enxurrada_acoes_corrt_realoc_pop','6_4_risco_destr_enxurrada_acoes_corrt_revegetacao',
'6_4_risco_destr_enxurrada_acoes_corrt_rev_rios_bacias','6_4_risco_destr_enxurrada_acoes_corrt_outras_solu','6_4_risco_destr_enxurrada_acoes_corrt_nenhuma',
'6_4_risco_destr_enxurrada_acoes_corrt_nao_sabe',]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_4_risco_destr_enxurrada_4anos_exist,214,27.61,30.78,113,25.11,368,20.51,636,38.13,462,38.79,133,28.48
6_4_risco_destr_enxurrada_maior_impac_edif,178,22.97,25.73,95,21.11,324,18.06,565,33.87,338,28.38,109,23.34
6_4_risco_destr_enxurrada_maior_impac_pess,116,14.97,16.90,72,16.00,246,13.71,390,23.38,192,16.12,40,8.57
6_4_risco_destr_enxurrada_maior_impac_obitos,13,1.68,1.55,9,2.00,20,1.11,42,2.52,14,1.18,1,0.21
6_4_risco_destr_enxurrada_area_nat_inund,124,16.00,19.13,65,14.44,213,11.87,426,25.54,288,24.18,72,15.42
6_4_risco_destr_enxurrada_area_nao_inund,61,7.87,9.76,39,8.67,110,6.13,211,12.65,152,12.76,31,6.64
6_4_risco_destr_enxurrada_area_ocp_regular,55,7.10,9.53,28,6.22,97,5.41,234,14.03,137,11.50,34,7.28
6_4_risco_destr_enxurrada_area_ocp_irregul,72,9.29,10.25,47,10.44,134,7.47,233,13.97,120,10.08,36,7.71
6_4_risco_destr_enxurrada_area_proc_erosiv,24,3.10,2.73,11,2.44,40,2.23,62,3.72,20,1.68,19,4.07
6_4_risco_destr_enxurrada_area_outros,27,3.48,2.93,12,2.67,45,2.51,55,3.30,40,3.36,11,2.36


#### Escorregamento ou deslizamento de encosta 

In [53]:
variaveis = ['6_5_risco_destr_deslizam_4anos_exist',#'6_5_risco_destr_deslizam_maior_impac_ano',
'6_5_risco_destr_deslizam_maior_impac_edif',#'6_5_risco_destr_deslizam_maior_impac_edif_area',
'6_5_risco_destr_deslizam_maior_impac_pess',#'6_5_risco_destr_deslizam_maior_impac_pess_area_deslj',
'6_5_risco_destr_deslizam_maior_impac_obitos',#'6_5_risco_destr_deslizam_maior_impac_obitos_area',
'6_5_risco_destr_deslizam_area_talude','6_5_risco_destr_deslizam_area_erosao_cronico','6_5_risco_destr_deslizam_area_sem_drenagem',
'6_5_risco_destr_deslizam_area_ocp_regular','6_5_risco_destr_deslizam_area_ocp_irregul','6_5_risco_destr_deslizam_area_sem_ocp',
'6_5_risco_destr_deslizam_area_outras_areas','6_5_risco_destr_deslizam_area_naosabe',
'6_5_risco_destr_deslizam_acoes_corrt_est_talude', '6_5_risco_destr_deslizam_acoes_corrt_retaludamento','6_5_risco_destr_deslizam_acoes_corrt_contencao',
'6_5_risco_destr_deslizam_acoes_corrt_macrodrenagens','6_5_risco_destr_deslizam_acoes_corrt_realoc_pop','6_5_risco_destr_deslizam_acoes_corrt_reveg_encosta',
'6_5_risco_destr_deslizam_acoes_corrt_outra_solucao','6_5_risco_destr_deslizam_acoes_corrt_nenhuma_interv','6_5_risco_destr_deslizam_acoes_corrt_nao_sabe',]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_5_risco_destr_deslizam_4anos_exist,88.00,11.35,16.81,62.00,13.78,175.00,9.75,482.00,28.90,185.00,15.53,31,6.64
6_5_risco_destr_deslizam_maior_impac_edif,67.00,8.65,12.15,48.00,10.67,140.00,7.80,369.00,22.12,99.00,8.31,20,4.28
6_5_risco_destr_deslizam_maior_impac_pess,55.00,7.10,9.77,41.00,9.11,119.00,6.63,312.00,18.71,63.00,5.29,9,1.93
6_5_risco_destr_deslizam_maior_impac_obitos,4.00,0.52,1.06,4.00,0.89,14.00,0.78,33.00,1.98,7.00,0.59,1,0.21
6_5_risco_destr_deslizam_area_talude,53.00,6.84,11.36,41.00,9.11,119.00,6.63,347.00,20.80,111.00,9.32,14,3.00
6_5_risco_destr_deslizam_area_erosao_cronico,28.00,3.61,3.61,19.00,4.22,44.00,2.45,107.00,6.41,23.00,1.93,8,1.71
6_5_risco_destr_deslizam_area_sem_drenagem,24.00,3.10,5.37,14.00,3.11,66.00,3.68,174.00,10.43,35.00,2.94,10,2.14
6_5_risco_destr_deslizam_area_ocp_regular,15.00,1.94,4.80,7.00,1.56,39.00,2.17,171.00,10.25,41.00,3.44,9,1.93
6_5_risco_destr_deslizam_area_ocp_irregul,33.00,4.26,7.44,25.00,5.56,95.00,5.30,232.00,13.91,54.00,4.53,8,1.71
6_5_risco_destr_deslizam_area_sem_ocp,12.00,1.55,4.08,8.00,1.78,29.00,1.62,127.00,7.61,59.00,4.95,4,0.86


#### Gerenciamento de riscos
- O município possui como instrumento de planejamento

In [54]:
variaveis = ['6_6_risco_destr_inst_planej_pde_enchente','6_6_risco_destr_inst_planej_lpuos_enchente','6_6_risco_destr_inst_planej_lei_esp_enchente',
'6_6_risco_destr_inst_planej_pde_deslizamento','6_6_risco_destr_inst_planej_lpuos_deslizamento','6_6_risco_destr_inst_planej_lei_esp_deslizamento',
'6_6_risco_destr_inst_planej_reducao_ruido','6_6_risco_destr_inst_planej_carta_geotecnica','6_6_risco_destr_inst_planej_plano_impln_obra',
'6_6_risco_destr_inst_planej_nenhum',
]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_6_risco_destr_inst_planej_pde_enchente,185,23.87,27.66,120,26.67,366,20.40,449,26.92,491,41.23,112,23.98
6_6_risco_destr_inst_planej_lpuos_enchente,199,25.68,28.26,117,26.00,410,22.85,429,25.72,479,40.22,136,29.12
6_6_risco_destr_inst_planej_lei_esp_enchente,27,3.48,4.03,14,3.11,52,2.90,81,4.86,56,4.70,21,4.50
6_6_risco_destr_inst_planej_pde_deslizamento,65,8.39,13.13,37,8.22,179,9.98,224,13.43,244,20.49,46,9.85
6_6_risco_destr_inst_planej_lpuos_deslizamento,90,11.61,13.89,51,11.33,181,10.09,222,13.31,264,22.17,54,11.56
6_6_risco_destr_inst_planej_lei_esp_deslizamento,11,1.42,1.92,5,1.11,25,1.39,37,2.22,30,2.52,10,2.14
6_6_risco_destr_inst_planej_reducao_ruido,66,8.52,13.11,44,9.78,169,9.42,302,18.11,178,14.95,36,7.71
6_6_risco_destr_inst_planej_carta_geotecnica,34,4.39,5.61,20,4.44,47,2.62,127,7.61,93,7.81,25,5.35
6_6_risco_destr_inst_planej_plano_impln_obra,40,5.16,6.13,26,5.78,94,5.24,119,7.13,75,6.30,27,5.78
6_6_risco_destr_inst_planej_nenhum,365,47.10,50.60,210,46.67,1047,58.36,854,51.20,449,37.70,256,54.82


- Em relação ao gerenciamento de riscos de desastres decorrentes de enchentes ou inundações graduais, ou enxurradas ou inundações bruscas, o município possui

In [55]:
variaveis = ['6_6_risco_destr_grisco_enchente_map_area_enchente','6_6_risco_destr_grisco_enchente_prog_hab_realocamento','6_6_risco_destr_grisco_enchente_fiscaliz_area_risco',
'6_6_risco_destr_grisco_enchente_plano_contigencia','6_6_risco_destr_grisco_enchente_projeto_eng','6_6_risco_destr_grisco_enchente_sistema_alerta',
'6_6_risco_destr_grisco_enchente_cadastro_risco','6_6_risco_destr_grisco_enchente_nenhum','6_6_risco_destr_grisco_enchente_limpeza_bueiro',
]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_6_risco_destr_grisco_enchente_map_area_enchente,243,31.35,38.92,170,37.78,453,25.25,796,47.72,641,53.82,104,22.27
6_6_risco_destr_grisco_enchente_prog_hab_realocamento,82,10.58,16.72,56,12.44,251,13.99,358,21.46,211,17.72,54,11.56
6_6_risco_destr_grisco_enchente_fiscaliz_area_risco,108,13.94,18.56,73,16.22,221,12.32,377,22.60,279,23.43,82,17.56
6_6_risco_destr_grisco_enchente_plano_contigencia,76,9.81,25.32,62,13.78,211,11.76,512,30.70,596,50.04,26,5.57
6_6_risco_destr_grisco_enchente_projeto_eng,35,4.52,6.00,22,4.89,76,4.24,130,7.79,73,6.13,33,7.07
6_6_risco_destr_grisco_enchente_sistema_alerta,29,3.74,7.84,21,4.67,84,4.68,193,11.57,132,11.08,6,1.28
6_6_risco_destr_grisco_enchente_cadastro_risco,131,16.90,15.39,100,22.22,211,11.76,316,18.94,191,16.04,38,8.14
6_6_risco_destr_grisco_enchente_nenhum,367,47.35,44.01,191,42.44,1055,58.81,637,38.19,277,23.26,290,62.10
6_6_risco_destr_grisco_enchente_limpeza_bueiro,420,54.19,68.44,242,53.78,1071,59.70,1331,79.80,880,73.89,283,60.60


- Em relação ao gerenciamento de riscos de desastres decorrentes de escorregamentos ou deslizamento de encostas, o município possui

In [56]:
variaveis = ['6_6_risco_destr_grisco_delizam_map_area_enchente','6_6_risco_destr_grisco_delizam_prog_hab_realocamento','6_6_risco_destr_grisco_delizam_fiscaliz_area_risco',
'6_6_risco_destr_grisco_delizam_plano_contigencia','6_6_risco_destr_grisco_delizam_projeto_eng','6_6_risco_destr_grisco_delizam_sistema_alerta',
'6_6_risco_destr_grisco_delizam_cadastro_risco','6_6_risco_destr_grisco_delizam_nenhum',
]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_6_risco_destr_grisco_delizam_map_area_enchente,127,16.39,25.14,93,20.67,298,16.61,592,35.49,375,31.49,40,8.57
6_6_risco_destr_grisco_delizam_prog_hab_realocamento,49,6.32,10.60,29,6.44,176,9.81,246,14.75,112,9.40,27,5.78
6_6_risco_destr_grisco_delizam_fiscaliz_area_risco,53,6.84,11.13,35,7.78,151,8.42,244,14.63,154,12.93,35,7.49
6_6_risco_destr_grisco_delizam_plano_contigencia,49,6.32,18.28,40,8.89,147,8.19,418,25.06,397,33.33,14,3.00
6_6_risco_destr_grisco_delizam_projeto_eng,21,2.71,4.39,14,3.11,57,3.18,112,6.71,43,3.61,18,3.85
6_6_risco_destr_grisco_delizam_sistema_alerta,9,1.16,4.42,7,1.56,52,2.90,127,7.61,54,4.53,6,1.28
6_6_risco_destr_grisco_delizam_cadastro_risco,77,9.94,10.89,59,13.11,143,7.97,246,14.75,134,11.25,24,5.14
6_6_risco_destr_grisco_delizam_nenhum,491,63.35,57.09,268,59.56,1213,67.61,808,48.44,524,44.00,364,77.94


- Em relação à gestão de riscos e de desastres, existe no município

In [57]:
variaveis = ['6_6_risco_destr_grisco_bombeiro','6_6_risco_destr_grisco_coord_defesa_civil',
'6_6_risco_destr_grisco_nucleo_defesa_civil','6_6_risco_destr_grisco_guarda_municp','6_6_risco_destr_grisco_nenhuma','6_6_risco_destr_grisco_naosabe',
]
analise_brasil(dfr_munic_gst_risco, variaveis)

,AM_LEGAL,AM_LEGAL (%),Media Pond,1 - Norte,1 - N (%),2 - Nordeste,2 - NE (%),3 - Sudeste,3 - SE (%),4 - Sul,4 - S (%),5 - Centro-Oeste,5 - CO (%)
6_6_risco_destr_grisco_bombeiro,120,15.48,19.36,79,17.56,157,8.75,352,21.10,392,32.91,96,20.56
6_6_risco_destr_grisco_coord_defesa_civil,407,52.52,76.17,268,59.56,1247,69.51,1385,83.03,1138,95.55,198,42.40
6_6_risco_destr_grisco_nucleo_defesa_civil,45,5.81,6.78,34,7.56,86,4.79,150,8.99,90,7.56,17,3.64
6_6_risco_destr_grisco_guarda_municp,160,20.65,22.56,88,19.56,643,35.84,388,23.26,99,8.31,38,8.14
6_6_risco_destr_grisco_nenhuma,188,24.26,14.60,96,21.33,276,15.38,214,12.83,26,2.18,201,43.04
6_6_risco_destr_grisco_naosabe,31,4.00,2.16,19,4.22,43,2.40,26,1.56,12,1.01,20,4.28
